Что делает этот код (кратко)

Degree-days: для каждой особи нимфального комплекса выбирается случайный total DD в диапазоне [233.9, 338.7], затем этот суммарный DD распределяется по пяти нимфальным стадиям согласно NYMPH_STAGE_PROP. Яйца — фиксированный EGG_DD. Для накопления используется простая модель: daily_dd = max(0, temp - base_temp); если накоплено >= требуемого — особь переходит в следующую стадию. Это даёт нужный разброс по погоде.

Детализация нимф: стадии nymph1…nymph5 реализованы как отдельные стадии с разными движениями и смертностями.

Дополнительная смертность: базовая смертность модифицируется погодой (экстремы: высокая температура, низкая влажность, низкая температура) и добавляется плотностно-зависимая смертность (каннибализм) — если количество соседей в радиусе ≈0.178 м превышает порог DENSITY_THRESHOLD, вероятность смертности растёт.

Добыча: генерируем 700 точек «добычи» внутри круга радиуса 10 м вокруг заданных координат (PREY_CENTER). Если хищник попадает в радиус обнаружения (10 м), его движение получает направленный вектор к ближайшей добыче (смешанный с случайностью — параметр PREY_BIAS).

Границы (GeoJSON): если передать путь к geojson (полигон), движение, которое ведёт за границы, запрещается (в текущей реализации — движение откатывается). При желании можно заменить откат на отскок или проекцию на границу (легко изменить).

Каждый агент = одна особь. (как вы просили)

Экспорт GeoTIFF по дням: после каждого дня создаётся GeoTIFF (density_day_XXX.tif) с плотностью агентов (пиксель = PIXEL_SIZE_M метров, задан в параметрах). Также экспортируем GeoJSON точек по дням (points_day_XXX.geojson). Оба формата открываются в QGIS напрямую.

Как калибровать / что ещё можно улучшить (быстрая шпаргалка)

Degree-day модель: текущая — простая (base temp, temp - base). Можно заменить на более точную методику (интеграл по часам, использование минимального и максимального порога).

Разделение пола: сейчас пол присваивается случайно; можно задать разные DD или смертности для самок/самцов.

Каннибализм: можно улучшить до реально событийного взаимодействия (например, вероятность поедания зависит от размеров стадий).

Ориентированное движение: можно вводить поле запаха/градиент добычи (растр), чтобы двигались не к отдельной точке, а к распределению ресурсов.

Производительность: при очень большом числе агентов (10k+) лучше применять агрегированную модель по ячейкам (cellular automata) или оптимизированные kd-tree расчёты.

Что сразу можно сделать дальше (я могу помочь)

Подготовить пример GeoJSON поля (ширина/высота) и протестировать ограничение границ.

Изменить стратегию поведения при достижении добычи (например, уменьшать количество добычи при поедании).

Переписать вывод для экспорта в QGIS как набор временных слоёв (один слой с атрибутом day) вместо множества GeoJSON/GeoTIFF.

Добавить почасовую временную сетку и более точный расчёт DD по часам (по часовому прогнозу OpenWeather).

In [15]:
import math
import random
import datetime
import json
import os
from typing import List, Tuple, Optional

import numpy as np
from shapely.geometry import Point, shape, Polygon
import rasterio
from rasterio.transform import from_origin
from tqdm import tqdm

# ---------------------------
#  Параметры симуляции
# ---------------------------

INIT_POINTS = [(52.134873, 79.278759)]
INIT_COUNTS = {"egg": 0, "nymph1": 0, "nymph2": 0, "nymph3": 1000, "nymph4": 0, "nymph5": 0, "adult": 0}

SIM_DAYS = 30

# Параметры DD / стадии клопа
NYMPH_DD_RANGE = (233.9, 338.7)
EGG_DD = 50.0
BASE_TEMP_EGG = 10.7
BASE_TEMP_NYMPH = 11.7
NYMPH_STAGE_PROP = np.array([0.10, 0.18, 0.20, 0.25, 0.27])
BASE_MORT = {"egg": 0.01, "nymph1": 0.01, "nymph2": 0.007, "nymph3": 0.005, "nymph4": 0.005, "nymph5": 0.005, "adult": 0.002}
DENSITY_AREA = 0.1
DENSITY_RADIUS = math.sqrt(DENSITY_AREA / math.pi)
DENSITY_THRESHOLD = 3
DENSITY_MORTALITY_FACTOR = 0.1
MOVE_BASE_METERS = {"egg": 0.0, "nymph1": 0.5, "nymph2": 2.0, "nymph3": 6.0, "nymph4": 10.0, "nymph5": 15.0, "adult": 25.0}
PREY_DETECTION_RADIUS = 10.0
PREY_BIAS = 0.8
UPPER_TEMP_THRESHOLD = 35.0
PIXEL_SIZE_M = 1.0

# Параметры роста жука
MIN_TEMP_FOR_GROWTH = 10.0
MAX_TEMP_FOR_GROWTH = 30.0
GROWTH_COEFF = 0.05  # a
PREY_CARRYING_CAPACITY = 3140  # пример
PREY_BASE_MORT = 0.01  # базовая дневная смертность жука

# ---------------------------
#  Гео-утилиты
# ---------------------------

def meters_to_deg_lat(m):
    return m / 111000.0

def meters_to_deg_lon(m, lat):
    return m / (111000.0 * math.cos(math.radians(lat)) + 1e-12)

def deg_lat_to_meters(dlat):
    return dlat * 111000.0

def deg_lon_to_meters(dlon, lat):
    return dlon * 111000.0 * math.cos(math.radians(lat))

def haversine_m(lat1, lon1, lat2, lon2):
    R = 6371000.0
    phi1 = math.radians(lat1); phi2 = math.radians(lat2)
    dphi = math.radians(lat2 - lat1); dlambda = math.radians(lon2 - lon1)
    a = math.sin(dphi/2)**2 + math.cos(phi1)*math.cos(phi2)*math.sin(dlambda/2)**2
    return 2*R*math.asin(math.sqrt(a))

# ---------------------------
#  Агент клопа
# ---------------------------

class Individual:
    def __init__(self, lat, lon, stage, sex=None):
        self.lat = float(lat)
        self.lon = float(lon)
        self.stage = stage
        self.alive = True
        self.dd_required = self.assign_dd_required(stage, sex)
        self.dd_accum = 0.0
        self.sex = sex if sex else random.choice(["F", "M"])
    
    def assign_dd_required(self, stage, sex):
        if stage == "egg":
            return EGG_DD
        if stage.startswith("nymph"):
            total = random.uniform(NYMPH_DD_RANGE[0], NYMPH_DD_RANGE[1])
            idx = int(stage.replace("nymph", "")) - 1
            return total * float(NYMPH_STAGE_PROP[idx])
        return float('inf')
    
    def progress(self, daily_dd):
        if not self.alive:
            return
        if self.stage == "adult":
            return
        self.dd_accum += daily_dd
        if self.dd_accum >= self.dd_required:
            if self.stage == "egg":
                self.stage = "nymph1"
            else:
                idx = int(self.stage.replace("nymph", ""))
                self.stage = f"nymph{idx+1}" if idx < 5 else "adult"
            self.dd_required = self.assign_dd_required(self.stage, self.sex)
            self.dd_accum = 0.0

# ---------------------------
#  Чтение метеоданных из архива
# ---------------------------

def read_weather_archive(filepath: str, start_date: datetime.date, days: int):
    """
    Читает файл архива вида:
      индекс;год;месяц;день; Tmin; Tavg; Tmax; осадки
    Возвращает список из days словарей {'dt': date, 'temp': float, 'precip': float}
    Если день отсутствует — ставит None или выбирает значения средние.
    """
    # читаем весь файл в словарь по дате
    data = {}
    with open(filepath, 'r', encoding='utf-8') as f:
        for line in f:
            parts = line.strip().split(';')
            if len(parts) < 8:
                continue
            _, yr, mo, da, tmin, tavg, tmax, precip = parts
            try:
                yr = int(yr); mo = int(mo); da = int(da)
                date = datetime.date(yr, mo, da)
                data[date] = {
                    "tmin": float(tmin),
                    "tavg": float(tavg),
                    "tmax": float(tmax),
                    "precip": float(precip)
                }
            except:
                continue
    result = []
    for d in range(days):
        cur = start_date + datetime.timedelta(days=d)
        rec = data.get(cur)
        if rec is not None:
            result.append({"dt": cur, "temp": rec["tavg"], "precip": rec["precip"]})
        else:
            # если отсутствует — поставить среднюю по всем или None
            # здесь — взять среднюю 20°C и 0 осадков
            result.append({"dt": cur, "temp": 20.0, "precip": 0.0})
    return result

# ---------------------------
#  Модель роста колорадского жука (добыча)
# ---------------------------

class PreyPopulation:
    def __init__(self, init_N: float, center_lat: float, center_lon: float, radius_m: float):
        self.N = init_N
        self.center_lat = center_lat
        self.center_lon = center_lon
        self.radius_m = radius_m
    
    def step(self, T_avg: float, precip: float):
        # базовая смертность
        self.N *= (1.0 - PREY_BASE_MORT)
        # вычислить r(T)
        if T_avg <= MIN_TEMP_FOR_GROWTH or T_avg >= MAX_TEMP_FOR_GROWTH:
            r = 0.0
        else:
            r = GROWTH_COEFF * (T_avg - MIN_TEMP_FOR_GROWTH)
        # поправка по осадкам (если мало осадков, уменьшить r)
        if precip < 1.0:  # порог осадков (мм)
            r *= 0.5
        # логистический рост
        growth_factor = math.exp(r)
        self.N = self.N * growth_factor * (1.0 - self.N / max(1.0, PREY_CARRYING_CAPACITY))
        if self.N < 0:
            self.N = 0.0
    
    def generate_points(self):
        # генерируем int(self.N) точек внутри круга radius_m
        pts = []
        count = int(round(self.N))
        for _ in range(count):
            r = self.radius_m * math.sqrt(random.random())
            theta = random.random() * 2 * math.pi
            dy_m = r * math.cos(theta)
            dx_m = r * math.sin(theta)
            lat = self.center_lat + meters_to_deg_lat(dy_m)
            lon = self.center_lon + meters_to_deg_lon(dx_m, self.center_lat)
            pts.append((lat, lon))
        return pts

# ---------------------------
#  Симулятор с учётом роста добычи
# ---------------------------

class PodisusSimulator:
    def __init__(self,
                 init_points: List[Tuple[float,float]],
                 init_counts: dict,
                 sim_days: int,
                 weather_filepath: str,
                 weather_start_date: str,
                 prey_init_N: float,
                 prey_radius_m: float,
                 geojson_path: Optional[str] = None):
        random.seed(42)
        np.random.seed(42)
        self.sim_days = sim_days
        self.agents: List[Individual] = []
        self.init_points = init_points

        for lat, lon in init_points:
            for stg, cnt in init_counts.items():
                for _ in range(cnt):
                    self.agents.append(Individual(lat, lon, stg))

        # настроим добычу
        # преобразуем weather_start_date
        sd = datetime.datetime.strptime(weather_start_date, "%d.%m.%Y").date()
        self.prey = PreyPopulation(init_N=prey_init_N, center_lat=init_points[0][0], center_lon=init_points[0][1], radius_m=prey_radius_m)

        # метеранные данные
        self.weather = read_weather_archive(weather_filepath, sd, sim_days)

        # границы поля (опционально)
        self.boundary_polygon: Optional[Polygon] = None
        if geojson_path and os.path.exists(geojson_path):
            with open(geojson_path, 'r', encoding='utf-8') as f:
                gj = json.load(f)
            geom = None
            if gj.get("type") == "FeatureCollection":
                polys = [shape(feat["geometry"]) for feat in gj["features"]]
                geom = polys[0].union(*polys[1:]) if len(polys) > 1 else polys[0]
            elif gj.get("type") == "Feature":
                geom = shape(gj["geometry"])
            else:
                geom = shape(gj)
            self.boundary_polygon = geom

        self.history = []
        os.makedirs("simulation_output", exist_ok=True)

    def run(self):
        for day in range(self.sim_days):
            w = self.weather[day]
            T = w["temp"]
            precip = w.get("precip", 0.0)
            # обновить добычу
            self.prey.step(T, precip)
            prey_pts = self.prey.generate_points()

            # симуляция клопов
            alive = [a for a in self.agents if a.alive]
            for a in alive:
                # смертность, развитие, движение — как прежде
                mort = BASE_MORT.get(a.stage, 0.01)
                if T > UPPER_TEMP_THRESHOLD:
                    mort *= 3.0
                if T < (BASE_TEMP_EGG if a.stage == "egg" else BASE_TEMP_NYMPH):
                    mort *= 1.8
                if random.random() < mort:
                    a.alive = False
                    continue

                # развитие
                base = BASE_TEMP_EGG if a.stage == "egg" else BASE_TEMP_NYMPH
                daily_dd = max(0.0, T - base)
                a.progress(daily_dd)

                # движение
                step_m = MOVE_BASE_METERS.get(a.stage, 0.0)
                if step_m <= 0:
                    continue

                # ориентир к добыче если в радиусе
                min_d = float('inf')
                target = None
                for (plat, plon) in prey_pts:
                    d = haversine_m(a.lat, a.lon, plat, plon)
                    if d < min_d:
                        min_d = d
                        target = (plat, plon)
                if target and min_d <= PREY_DETECTION_RADIUS:
                    vy = deg_lat_to_meters(target[0] - a.lat)
                    vx = deg_lon_to_meters(target[1] - a.lon, a.lat)
                    norm = math.hypot(vx, vy) + 1e-12
                    ux, uy = vx/norm, vy/norm
                    theta = random.random() * 2*math.pi
                    rx, ry = math.cos(theta), math.sin(theta)
                    vx_m = PREY_BIAS * ux + (1 - PREY_BIAS) * rx
                    vy_m = PREY_BIAS * uy + (1 - PREY_BIAS) * ry
                    norm2 = math.hypot(vx_m, vy_m) + 1e-12
                    vx_m = vx_m / norm2 * step_m
                    vy_m = vy_m / norm2 * step_m
                else:
                    theta = random.random() * 2*math.pi
                    vx_m = step_m * math.cos(theta)
                    vy_m = step_m * math.sin(theta)

                dlat = meters_to_deg_lat(vy_m)
                dlon = meters_to_deg_lon(vx_m, a.lat)
                new_lat = a.lat + dlat
                new_lon = a.lon + dlon

                if self.boundary_polygon is not None:
                    pt = Point(new_lon, new_lat)
                    if not self.boundary_polygon.contains(pt):
                        new_lat, new_lon = a.lat, a.lon

                a.lat, a.lon = new_lat, new_lon

            day_points = [(a.lat, a.lon, a.stage) for a in self.agents if a.alive]
            self.history.append(day_points)

            self.export_geojson_day(day, day_points)
            self.export_geotiff_day(day, day_points)

    def export_geojson_day(self, d, pts):
        feats = []
        for (lat, lon, stg) in pts:
            feats.append({"type":"Feature", "geometry":{"type":"Point","coordinates":[lon, lat]}, "properties":{"stage":stg}})
        fc = {"type":"FeatureCollection", "features":feats}
        with open(f"simulation_output/points_day_{d:03d}.geojson", "w", encoding="utf-8") as f:
            json.dump(fc, f, ensure_ascii=False, indent=2)

    def export_geotiff_day(self, d, pts):
        if not pts:
            return
        lats = [p[0] for p in pts]; lons = [p[1] for p in pts]
        min_lat, max_lat = min(lats), max(lats); min_lon, max_lon = min(lons), max(lons)
        pad = 20.0
        padlat = meters_to_deg_lat(pad); padlon = meters_to_deg_lon(pad, (min_lat+max_lat)/2.0)
        min_lat -= padlat; max_lat += padlat; min_lon -= padlon; max_lon += padlon

        center_lat = (min_lat + max_lat)/2.0
        x_min_m = deg_lon_to_meters(min_lon, center_lat)
        x_max_m = deg_lon_to_meters(max_lon, center_lat)
        y_min_m = deg_lat_to_meters(min_lat)
        y_max_m = deg_lat_to_meters(max_lat)
        width_m = x_max_m - x_min_m; height_m = y_max_m - y_min_m
        if width_m <= 0 or height_m <= 0:
            return
        px = max(1, int(math.ceil(width_m / PIXEL_SIZE_M)))
        py = max(1, int(math.ceil(height_m / PIXEL_SIZE_M)))

        pix_h = meters_to_deg_lat(PIXEL_SIZE_M); pix_w = meters_to_deg_lon(PIXEL_SIZE_M, center_lat)
        transform = from_origin(min_lon, max_lat, pix_w, pix_h)

        raster = np.zeros((py, px), dtype=np.uint16)
        for lat, lon, _ in pts:
            col = int((lon - min_lon) / pix_w)
            row = int((max_lat - lat) / pix_h)
            if 0 <= row < py and 0 <= col < px:
                raster[row, col] += 1

        with rasterio.open(f"simulation_output/density_day_{d:03d}.tif", 'w',
                           driver='GTiff', height=py, width=px,
                           count=1, dtype=raster.dtype,
                           crs="EPSG:4326", transform=transform, compress='lzw') as dst:
            dst.write(raster, 1)



In [18]:
# ---------------------------
#  Параметры симуляции
# ---------------------------

INIT_POINTS = [(52.134873, 79.278759)]
INIT_COUNTS = {"egg": 0, "nymph1": 0, "nymph2": 0, "nymph3": 1000, "nymph4": 0, "nymph5": 0, "adult": 0}

SIM_DAYS = 30

# Параметры DD / стадии клопа
NYMPH_DD_RANGE = (233.9, 338.7)
EGG_DD = 50.0
BASE_TEMP_EGG = 10.7
BASE_TEMP_NYMPH = 11.7
NYMPH_STAGE_PROP = np.array([0.10, 0.18, 0.20, 0.25, 0.27])
BASE_MORT = {"egg": 0.01, "nymph1": 0.01, "nymph2": 0.007, "nymph3": 0.005, "nymph4": 0.005, "nymph5": 0.005, "adult": 0.002}
DENSITY_AREA = 0.1
DENSITY_RADIUS = math.sqrt(DENSITY_AREA / math.pi)
DENSITY_THRESHOLD = 3
DENSITY_MORTALITY_FACTOR = 0.1
MOVE_BASE_METERS = {"egg": 0.0, "nymph1": 0.5, "nymph2": 2.0, "nymph3": 6.0, "nymph4": 10.0, "nymph5": 15.0, "adult": 25.0}
PREY_DETECTION_RADIUS = 10.0
PREY_BIAS = 0.8
UPPER_TEMP_THRESHOLD = 35.0
PIXEL_SIZE_M = 1.0

# Параметры роста жука
MIN_TEMP_FOR_GROWTH = 10.0
MAX_TEMP_FOR_GROWTH = 30.0
GROWTH_COEFF = 0.05  # a
PREY_CARRYING_CAPACITY = 3140  # пример
PREY_BASE_MORT = 0.01  # базовая дневная смертность жука

In [19]:
# ---------------------------
#  Основной запуск
# ---------------------------

if __name__ == "__main__":
    weather_file = r"C:\Users\pochy\Desktop\Стартап Экологические услуги\Модель расселения\wr352396a1\wr352396a1.txt"
    start_date = datetime.datetime.strptime("15.06.2024", "%d.%m.%Y").date()
    sim = PodisusSimulator(
        init_points = INIT_POINTS,
        init_counts = INIT_COUNTS,
        sim_days = SIM_DAYS,
        weather_filepath = weather_file,
        weather_start_date = "15.06.2024",
        prey_init_N = 700,
        prey_radius_m = 10.0,
        geojson_path = None
    )
    sim.run()
    print("Done. Outputs in simulation_output/")


Done. Outputs in simulation_output/


Отдельный эксперимент по дополнению

In [25]:

import math
import random
import datetime
import json
import os
from typing import List, Tuple, Optional, Dict, Any

import numpy as np
from shapely.geometry import Point, shape, Polygon
import rasterio
from rasterio.transform import from_origin
from tqdm import tqdm

# ---------------------------
#  Параметры симуляции
# ---------------------------

# Инициализация начальных точек и количества особей
INIT_POINTS = [(52.134873, 79.278759)]
INIT_COUNTS_PODISUS = {"egg": 0, "nymph1": 0, "nymph2": 0, "nymph3": 0, "nymph4": 0, "nymph5": 0, "adult": 1000}
INIT_COUNTS_LEPTINOTARSA = {"egg": 0, "larva1": 0, "larva2": 0, "larva3": 0, "larva4": 0, "pupa": 0, "adult": 500}

SIM_DAYS = 30

# --- Параметры Podisus maculiventris (хищный клоп) ---
# DD (Degree Days)
NYMPH_DD_RANGE_PODISUS = (233.9, 338.7)  # Общий диапазон для нимф
EGG_DD_PODISUS = 50.0
BASE_TEMP_EGG_PODISUS = 10.7
BASE_TEMP_NYMPH_PODISUS = 11.7
NYMPH_STAGE_PROP_PODISUS = np.array([0.10, 0.18, 0.20, 0.25, 0.27])
# Смертность
BASE_MORT_PODISUS = {
    "egg": 0.01, "nymph1": 0.01, "nymph2": 0.007, "nymph3": 0.005, "nymph4": 0.005, "nymph5": 0.005, "adult": 0.002
}
# Движение
MOVE_BASE_METERS_PODISUS = {
    "egg": 0.0, "nymph1": 0.5, "nymph2": 2.0, "nymph3": 6.0, "nymph4": 10.0, "nymph5": 15.0, "adult": 25.0
}
# Хищничество
PREY_DETECTION_RADIUS_PODISUS = 10.0
PREY_BIAS_PODISUS = 0.8
# Температура
UPPER_TEMP_THRESHOLD_PODISUS = 35.0
LOWER_TEMP_THRESHOLD_PODISUS = 10.0 # Пример, может отличаться
# Размножение
MIN_AGE_ADULT_PODISUS = 5 # дней до первой кладки

# --- Параметры Leptinotarsa decemlineata (колорадский жук) ---
# DD (Degree Days)
BASE_TEMP_EGG_LEPTINO = 11.5
BASE_TEMP_LARVA_LEPTINO = 11.5
BASE_TEMP_PUPA_LEPTINO = 11.5
BASE_TEMP_ADULT_LEPTINO = 11.5
# Примерные DD для развития (взяты из общих данных)
DD_EGG_LEPTINO = 40.0 # Пример
DD_LARVA_TOTAL_LEPTINO = 150.0 # Пример, сумма для всех 4 стадий
DD_PUPA_LEPTINO = 100.0 # Пример
DD_PER_LARVA_STAGE_LEPTINO = DD_LARVA_TOTAL_LEPTINO / 4
# Смертность
BASE_MORT_LEPTINO = {
    "egg": 0.05, "larva1": 0.1, "larva2": 0.1, "larva3": 0.1, "larva4": 0.1, "pupa": 0.05, "adult": 0.005
}
# Движение
MOVE_BASE_METERS_LEPTINO = {
    "egg": 0.0, "larva1": 0.1, "larva2": 0.2, "larva3": 0.5, "larva4": 1.0, "pupa": 0.0, "adult": 10.0
}
# Потребление картофеля (условные единицы)
CONSUMPTION_LARVA_LEPTINO = [0.24, 0.50, 2.04, 4.15] # см2/день по стадиям 1-4
CONSUMPTION_ADULT_LEPTINO = 10.0 # см2/день
# Размножение
MIN_AGE_ADULT_LEPTINO = 5 # дней до первой кладки
CLUTCH_SIZE_LEPTINO = 30 # среднее количество яиц в кладке
FECUNDITY_LEPTINO = 10 # примерное количество кладок за жизнь
# Температура
UPPER_TEMP_THRESHOLD_LEPTINO = 33.0 # Пример
LOWER_TEMP_THRESHOLD_LEPTINO = 10.0 # Пример

# --- Общие параметры ---
PIXEL_SIZE_M = 1.0
DENSITY_RADIUS = math.sqrt(0.1 / math.pi) # Радиус для плотности 0.1 м2
DENSITY_THRESHOLD = 3
DENSITY_MORTALITY_FACTOR = 0.1

# ---------------------------
#  Гео-утилиты
# ---------------------------

def meters_to_deg_lat(m):
    return m / 111000.0

def meters_to_deg_lon(m, lat):
    return m / (111000.0 * math.cos(math.radians(lat)) + 1e-12)

def deg_lat_to_meters(dlat):
    return dlat * 111000.0

def deg_lon_to_meters(dlon, lat):
    return dlon * 111000.0 * math.cos(math.radians(lat))

def haversine_m(lat1, lon1, lat2, lon2):
    R = 6371000.0
    phi1 = math.radians(lat1); phi2 = math.radians(lat2)
    dphi = math.radians(lat2 - lat1); dlambda = math.radians(lon2 - lon1)
    a = math.sin(dphi/2)**2 + math.cos(phi1)*math.cos(phi2)*math.sin(dlambda/2)**2
    return 2*R*math.asin(math.sqrt(a))

# ---------------------------
#  Агент Podisus maculiventris
# ---------------------------

class PodisusIndividual:
    def __init__(self, lat, lon, stage, sex=None):
        self.lat = float(lat)
        self.lon = float(lon)
        self.stage = stage
        self.alive = True
        self.dd_required = self.assign_dd_required(stage)
        self.dd_accum = 0.0
        self.sex = sex if sex else random.choice(["F", "M"])
        self.age_days = 0 # Возраст в днях
        self.last_mated = -100 # Дней с последнего спаривания
    
    def assign_dd_required(self, stage):
        if stage == "egg":
            return EGG_DD_PODISUS
        if stage.startswith("nymph"):
            total = random.uniform(NYMPH_DD_RANGE_PODISUS[0], NYMPH_DD_RANGE_PODISUS[1])
            idx = int(stage.replace("nymph", "")) - 1
            return total * float(NYMPH_STAGE_PROP_PODISUS[idx])
        # Для взрослых DD не накапливается
        return float('inf')

    def progress(self, daily_dd):
        if not self.alive or self.stage == "adult":
            return
        self.dd_accum += daily_dd
        if self.dd_accum >= self.dd_required:
            if self.stage == "egg":
                self.stage = "nymph1"
            elif self.stage.startswith("nymph"):
                idx = int(self.stage.replace("nymph", ""))
                self.stage = f"nymph{idx+1}" if idx < 5 else "adult"
            self.dd_required = self.assign_dd_required(self.stage)
            self.dd_accum = 0.0

# ---------------------------
#  Агент Leptinotarsa decemlineata
# ---------------------------

class LeptinotarsaIndividual:
    def __init__(self, lat, lon, stage, sex=None):
        self.lat = float(lat)
        self.lon = float(lon)
        self.stage = stage
        self.alive = True
        self.dd_required = self.assign_dd_required(stage)
        self.dd_accum = 0.0
        self.sex = sex if sex else random.choice(["F", "M"])
        self.age_days = 0 # Возраст в днях
        self.last_mated = -100 # Дней с последнего спаривания

    def assign_dd_required(self, stage):
        if stage == "egg":
            return DD_EGG_LEPTINO
        if stage.startswith("larva"):
            return DD_PER_LARVA_STAGE_LEPTINO
        if stage == "pupa":
            return DD_PUPA_LEPTINO
        # Для взрослых DD не накапливается
        return float('inf')

    def progress(self, daily_dd):
        if not self.alive or self.stage == "adult":
            return
        self.dd_accum += daily_dd
        if self.dd_accum >= self.dd_required:
            if self.stage == "egg":
                self.stage = "larva1"
            elif self.stage.startswith("larva"):
                idx = int(self.stage.replace("larva", ""))
                self.stage = f"larva{idx+1}" if idx < 4 else "pupa"
            elif self.stage == "pupa":
                self.stage = "adult"
            self.dd_required = self.assign_dd_required(self.stage)
            self.dd_accum = 0.0

# ---------------------------
#  Чтение метеоданных из архива
# ---------------------------

def read_weather_archive(filepath: str, start_date: datetime.date, days: int):
    """
    Читает файл архива вида:
      индекс;год;месяц;день; Tmin; Tavg; Tmax; осадки
    Возвращает список из days словарей {'dt': date, 'temp': float, 'precip': float}
    """
    data = {}
    with open(filepath, 'r', encoding='utf-8') as f:
        for line in f:
            parts = line.strip().split(';')
            if len(parts) < 8:
                continue
            _, yr, mo, da, tmin, tavg, tmax, precip = parts
            try:
                yr = int(yr); mo = int(mo); da = int(da)
                date = datetime.date(yr, mo, da)
                data[date] = {
                    "tmin": float(tmin),
                    "tavg": float(tavg),
                    "tmax": float(tmax),
                    "precip": float(precip)
                }
            except:
                continue
    result = []
    for d in range(days):
        cur = start_date + datetime.timedelta(days=d)
        rec = data.get(cur)
        if rec is not None:
            result.append({"dt": cur, "temp": rec["tavg"], "precip": rec["precip"]})
        else:
            # если отсутствует — поставить среднюю по всем или None
            result.append({"dt": cur, "temp": 20.0, "precip": 0.0})
    return result

# ---------------------------
#  Симулятор
# ---------------------------

class CombinedSimulator:
    def __init__(self,
                 init_points: List[Tuple[float,float]],
                 init_counts_podisus: dict,
                 init_counts_leptinotarsa: dict,
                 sim_days: int,
                 weather_filepath: str,
                 weather_start_date: str,
                 geojson_path: Optional[str] = None):
        random.seed(42)
        np.random.seed(42)
        self.sim_days = sim_days
        self.agents_podisus: List[PodisusIndividual] = []
        self.agents_leptinotarsa: List[LeptinotarsaIndividual] = []
        self.init_points = init_points

        # Инициализация Podisus
        for lat, lon in init_points:
            for stg, cnt in init_counts_podisus.items():
                for _ in range(cnt):
                    self.agents_podisus.append(PodisusIndividual(lat, lon, stg))

        # Инициализация Leptinotarsa
        for lat, lon in init_points:
            for stg, cnt in init_counts_leptinotarsa.items():
                for _ in range(cnt):
                    self.agents_leptinotarsa.append(LeptinotarsaIndividual(lat, lon, stg))

        # Метеоданные
        sd = datetime.datetime.strptime(weather_start_date, "%d.%m.%Y").date()
        self.weather = read_weather_archive(weather_filepath, sd, sim_days)

        # Границы поля (опционально)
        self.boundary_polygon: Optional[Polygon] = None
        if geojson_path and os.path.exists(geojson_path):
            with open(geojson_path, 'r', encoding='utf-8') as f:
                gj = json.load(f)
            geom = None
            if gj.get("type") == "FeatureCollection":
                polys = [shape(feat["geometry"]) for feat in gj["features"]]
                geom = polys[0].union(*polys[1:]) if len(polys) > 1 else polys[0]
            elif gj.get("type") == "Feature":
                geom = shape(gj["geometry"])
            else:
                geom = shape(gj)
            self.boundary_polygon = geom

        self.history_podisus = []
        self.history_leptinotarsa = []
        os.makedirs("simulation_output", exist_ok=True)

    def run(self):
        for day in tqdm(range(self.sim_days), desc="Simulating days"):
            w = self.weather[day]
            T = w["temp"]
            precip = w.get("precip", 0.0)

            # --- Обновление Podisus ---
            alive_podisus = [a for a in self.agents_podisus if a.alive]
            for a in alive_podisus:
                a.age_days += 1
                # --- Смертность ---
                mort = BASE_MORT_PODISUS.get(a.stage, 0.01)
                # Температурная смертность
                if T > UPPER_TEMP_THRESHOLD_PODISUS or T < LOWER_TEMP_THRESHOLD_PODISUS:
                    mort *= 2.0 # Увеличение смертности при экстремальных температурах
                # Плотностная смертность (пример для нимф)
                if a.stage.startswith("nymph"):
                    nearby = [x for x in alive_podisus if x != a and x.alive and haversine_m(a.lat, a.lon, x.lat, x.lon) <= DENSITY_RADIUS]
                    if len(nearby) >= DENSITY_THRESHOLD:
                        mort += len(nearby) * DENSITY_MORTALITY_FACTOR
                if random.random() < mort:
                    a.alive = False
                    continue

                # --- Развитие ---
                base_temp = BASE_TEMP_EGG_PODISUS if a.stage == "egg" else BASE_TEMP_NYMPH_PODISUS
                daily_dd = max(0.0, T - base_temp)
                a.progress(daily_dd)

                # --- Размножение ---
                if a.stage == "adult" and a.sex == "F" and a.age_days >= MIN_AGE_ADULT_PODISUS:
                    # Простая модель размножения
                    if random.random() < 0.1: # 10% шанс на размножение в день
                        # Создание яиц
                        for _ in range(10): # Пример количества яиц в кладке
                            new_lat = a.lat + meters_to_deg_lat(random.uniform(-1, 1))
                            new_lon = a.lon + meters_to_deg_lon(random.uniform(-1, 1), a.lat)
                            self.agents_podisus.append(PodisusIndividual(new_lat, new_lon, "egg"))

                # --- Движение ---
                step_m = MOVE_BASE_METERS_PODISUS.get(a.stage, 0.0)
                if step_m > 0:
                    # Поиск ближайшей добычи (Leptinotarsa)
                    min_d = float('inf')
                    target = None
                    for prey_agent in self.agents_leptinotarsa:
                        if prey_agent.alive:
                            d = haversine_m(a.lat, a.lon, prey_agent.lat, prey_agent.lon)
                            if d < min_d:
                                min_d = d
                                target = (prey_agent.lat, prey_agent.lon)
                    
                    # Движение к добыче или случайное
                    if target and min_d <= PREY_DETECTION_RADIUS_PODISUS:
                        vy = deg_lat_to_meters(target[0] - a.lat)
                        vx = deg_lon_to_meters(target[1] - a.lon, a.lat)
                        norm = math.hypot(vx, vy) + 1e-12
                        ux, uy = vx/norm, vy/norm
                        theta = random.random() * 2*math.pi
                        rx, ry = math.cos(theta), math.sin(theta)
                        vx_m = PREY_BIAS_PODISUS * ux + (1 - PREY_BIAS_PODISUS) * rx
                        vy_m = PREY_BIAS_PODISUS * uy + (1 - PREY_BIAS_PODISUS) * ry
                        norm2 = math.hypot(vx_m, vy_m) + 1e-12
                        vx_m = vx_m / norm2 * step_m
                        vy_m = vy_m / norm2 * step_m
                    else:
                        theta = random.random() * 2*math.pi
                        vx_m = step_m * math.cos(theta)
                        vy_m = step_m * math.sin(theta)

                    dlat = meters_to_deg_lat(vy_m)
                    dlon = meters_to_deg_lon(vx_m, a.lat)
                    new_lat = a.lat + dlat
                    new_lon = a.lon + dlon

                    if self.boundary_polygon is not None:
                        pt = Point(new_lon, new_lat)
                        if not self.boundary_polygon.contains(pt):
                            new_lat, new_lon = a.lat, a.lon

                    a.lat, a.lon = new_lat, new_lon

            # --- Обновление Leptinotarsa ---
            alive_leptino = [a for a in self.agents_leptinotarsa if a.alive]
            for a in alive_leptino:
                a.age_days += 1
                # --- Смертность ---
                mort = BASE_MORT_LEPTINO.get(a.stage, 0.01)
                # Температурная смертность
                if T > UPPER_TEMP_THRESHOLD_LEPTINO or T < LOWER_TEMP_THRESHOLD_LEPTINO:
                    mort *= 2.0
                # Плотностная смертность (пример для личинок)
                if a.stage.startswith("larva"):
                    nearby = [x for x in alive_leptino if x != a and x.alive and haversine_m(a.lat, a.lon, x.lat, x.lon) <= DENSITY_RADIUS]
                    if len(nearby) >= DENSITY_THRESHOLD:
                        mort += len(nearby) * DENSITY_MORTALITY_FACTOR
                # Смертность от хищничества (Podisus)
                if a.stage.startswith("larva"): # Личинки уязвимы
                    nearby_predators = [p for p in alive_podisus if haversine_m(a.lat, a.lon, p.lat, p.lon) <= PREY_DETECTION_RADIUS_PODISUS]
                    if nearby_predators:
                        # Простая модель: 50% шанс смерти если хищник рядом
                        if random.random() < 0.5:
                            a.alive = False
                            continue
                if random.random() < mort:
                    a.alive = False
                    continue

                # --- Развитие ---
                base_temp = BASE_TEMP_EGG_LEPTINO if a.stage == "egg" else BASE_TEMP_LARVA_LEPTINO if a.stage.startswith("larva") else BASE_TEMP_PUPA_LEPTINO if a.stage == "pupa" else BASE_TEMP_ADULT_LEPTINO
                daily_dd = max(0.0, T - base_temp)
                a.progress(daily_dd)

                # --- Размножение ---
                if a.stage == "adult" and a.sex == "F" and a.age_days >= MIN_AGE_ADULT_LEPTINO:
                    # Простая модель размножения
                    if random.random() < 0.05: # 5% шанс на размножение в день
                        # Создание яиц
                        for _ in range(CLUTCH_SIZE_LEPTINO): # Пример количества яиц в кладке
                            new_lat = a.lat + meters_to_deg_lat(random.uniform(-1, 1))
                            new_lon = a.lon + meters_to_deg_lon(random.uniform(-1, 1), a.lat)
                            self.agents_leptinotarsa.append(LeptinotarsaIndividual(new_lat, new_lon, "egg"))

                # --- Движение ---
                step_m = MOVE_BASE_METERS_LEPTINO.get(a.stage, 0.0)
                if step_m > 0:
                    theta = random.random() * 2*math.pi
                    vx_m = step_m * math.cos(theta)
                    vy_m = step_m * math.sin(theta)

                    dlat = meters_to_deg_lat(vy_m)
                    dlon = meters_to_deg_lon(vx_m, a.lat)
                    new_lat = a.lat + dlat
                    new_lon = a.lon + dlon

                    if self.boundary_polygon is not None:
                        pt = Point(new_lon, new_lat)
                        if not self.boundary_polygon.contains(pt):
                            new_lat, new_lon = a.lat, a.lon

                    a.lat, a.lon = new_lat, new_lon

            # Сохранение истории
            day_points_podisus = [(a.lat, a.lon, a.stage) for a in self.agents_podisus if a.alive]
            day_points_leptino = [(a.lat, a.lon, a.stage) for a in self.agents_leptinotarsa if a.alive]
            
            self.history_podisus.append(day_points_podisus)
            self.history_leptinotarsa.append(day_points_leptino)

            self.export_geojson_day(day, day_points_podisus, "podisus")
            self.export_geojson_day(day, day_points_leptino, "leptinotarsa")
            self.export_geotiff_day(day, day_points_podisus, "podisus")
            self.export_geotiff_day(day, day_points_leptino, "leptinotarsa")

    def export_geojson_day(self, d, pts, species_name):
        feats = []
        for (lat, lon, stg) in pts:
            feats.append({
                "type":"Feature",
                "geometry":{"type":"Point","coordinates":[lon, lat]},
                "properties":{"stage":stg, "species": species_name}
            })
        fc = {"type":"FeatureCollection", "features":feats}
        filename = f"simulation_output/{species_name}_points_day_{d:03d}.geojson"
        with open(filename, "w", encoding="utf-8") as f:
            json.dump(fc, f, ensure_ascii=False, indent=2)

    def export_geotiff_day(self, d, pts, species_name):
        if not pts:
            return
        lats = [p[0] for p in pts]; lons = [p[1] for p in pts]
        min_lat, max_lat = min(lats), max(lats); min_lon, max_lon = min(lons), max(lons)
        pad = 20.0
        padlat = meters_to_deg_lat(pad); padlon = meters_to_deg_lon(pad, (min_lat+max_lat)/2.0)
        min_lat -= padlat; max_lat += padlat; min_lon -= padlon; max_lon += padlon

        center_lat = (min_lat + max_lat)/2.0
        x_min_m = deg_lon_to_meters(min_lon, center_lat)
        x_max_m = deg_lon_to_meters(max_lon, center_lat)
        y_min_m = deg_lat_to_meters(min_lat)
        y_max_m = deg_lat_to_meters(max_lat)
        width_m = x_max_m - x_min_m; height_m = y_max_m - y_min_m
        if width_m <= 0 or height_m <= 0:
            return
        px = max(1, int(math.ceil(width_m / PIXEL_SIZE_M)))
        py = max(1, int(math.ceil(height_m / PIXEL_SIZE_M)))

        pix_h = meters_to_deg_lat(PIXEL_SIZE_M); pix_w = meters_to_deg_lon(PIXEL_SIZE_M, center_lat)
        transform = from_origin(min_lon, max_lat, pix_w, pix_h)

        raster = np.zeros((py, px), dtype=np.uint16)
        for lat, lon, _ in pts:
            col = int((lon - min_lon) / pix_w)
            row = int((max_lat - lat) / pix_h)
            if 0 <= row < py and 0 <= col < px:
                raster[row, col] += 1

        filename = f"simulation_output/{species_name}_density_day_{d:03d}.tif"
        with rasterio.open(filename, 'w',
                           driver='GTiff', height=py, width=px,
                           count=1, dtype=raster.dtype,
                           crs="EPSG:4326", transform=transform, compress='lzw') as dst:
            dst.write(raster, 1)

# ---------------------------
#  Основной запуск
# ---------------------------

if __name__ == "__main__":
    weather_file = r"C:\Users\pochy\Desktop\Стартап Экологические услуги\Модель расселения\wr352396a1\wr352396a1.txt"
    start_date = datetime.datetime.strptime("15.06.2024", "%d.%m.%Y").date()
    sim = CombinedSimulator(
        init_points = INIT_POINTS,
        init_counts_podisus = INIT_COUNTS_PODISUS,
        init_counts_leptinotarsa = INIT_COUNTS_LEPTINOTARSA,
        sim_days = SIM_DAYS,
        weather_filepath = weather_file,
        weather_start_date = "15.06.2024",
        geojson_path = None
    )
    sim.run()
    print("Done. Outputs in simulation_output/")


Simulating days:  83%|████████▎ | 25/30 [17:41<03:32, 42.47s/it]


KeyboardInterrupt: 

In [1]:
import math
import random
import datetime
import json
import os
from typing import List, Tuple, Optional, Dict, Any

import numpy as np
from shapely.geometry import Point, shape, Polygon
import rasterio
from rasterio.transform import from_origin
from tqdm import tqdm

# ---------------------------
#  Параметры симуляции
# ---------------------------

# Инициализация начальных точек и количества особей (Podisus)
INIT_POINTS = [(52.134873, 79.278759)]
INIT_COUNTS_PODISUS = {"egg": 0, "nymph1": 0, "nymph2": 0, "nymph3": 1000, "nymph4": 0, "nymph5": 0, "adult": 0}
INIT_COUNTS_LEPTINOTARSA = {"egg": 0, "larva1": 0, "larva2": 0, "larva3": 0, "larva4": 2000, "pupa": 0, "adult": 0}

SIM_DAYS = 30

# --- Параметры Podisus maculiventris (хищный клоп) ---
# DD (Degree Days)
NYMPH_DD_RANGE_PODISUS = (233.9, 338.7)
EGG_DD_PODISUS = 50.0
BASE_TEMP_EGG_PODISUS = 10.7
BASE_TEMP_NYMPH_PODISUS = 11.7
NYMPH_STAGE_PROP_PODISUS = np.array([0.10, 0.18, 0.20, 0.25, 0.27])
BASE_MORT_PODISUS = {
    "egg": 0.01, "nymph1": 0.01, "nymph2": 0.007, "nymph3": 0.005, "nymph4": 0.005, "nymph5": 0.005, "adult": 0.002
}
MOVE_BASE_METERS_PODISUS = {
    "egg": 0.0, "nymph1": 0.5, "nymph2": 2.0, "nymph3": 6.0, "nymph4": 10.0, "nymph5": 15.0, "adult": 25.0
}
PREY_DETECTION_RADIUS_PODISUS = 10.0
PREY_BIAS_PODISUS = 0.8
UPPER_TEMP_THRESHOLD_PODISUS = 35.0
LOWER_TEMP_THRESHOLD_PODISUS = 10.0
MIN_AGE_ADULT_PODISUS = 5

# --- Параметры Leptinotarsa decemlineata (колорадский жук) ---
BASE_TEMP_EGG_LEPTINO = 11.5
BASE_TEMP_LARVA_LEPTINO = 11.5
BASE_TEMP_PUPA_LEPTINO = 11.5
BASE_TEMP_ADULT_LEPTINO = 11.5
DD_EGG_LEPTINO = 40.0
DD_LARVA_TOTAL_LEPTINO = 150.0
DD_PUPA_LEPTINO = 100.0
DD_PER_LARVA_STAGE_LEPTINO = DD_LARVA_TOTAL_LEPTINO / 4
BASE_MORT_LEPTINO = {
    "egg": 0.05, "larva1": 0.1, "larva2": 0.1, "larva3": 0.1, "larva4": 0.1, "pupa": 0.05, "adult": 0.005
}
MOVE_BASE_METERS_LEPTINO = {
    "egg": 0.0, "larva1": 0.1, "larva2": 0.2, "larva3": 0.5, "larva4": 1.0, "pupa": 0.0, "adult": 10.0
}
CONSUMPTION_LARVA_LEPTINO = [0.24, 0.50, 2.04, 4.15] # см2/день по стадиям 1-4
CONSUMPTION_ADULT_LEPTINO = 10.0 # см2/день
MIN_AGE_ADULT_LEPTINO = 5
CLUTCH_SIZE_LEPTINO = 30
FECUNDITY_LEPTINO = 10
UPPER_TEMP_THRESHOLD_LEPTINO = 33.0
LOWER_TEMP_THRESHOLD_LEPTINO = 10.0

# --- Общие параметры ---
PIXEL_SIZE_M = 1.0
DENSITY_RADIUS = math.sqrt(0.1 / math.pi)
DENSITY_THRESHOLD = 3
DENSITY_MORTALITY_FACTOR = 0.1

# --- НОВОЕ: Дата начала года для DD ---
YEAR_START_DATE = datetime.date(2024, 1, 1) # <- Задается вручную
MANUAL_PREY_POINTS = [
    (52.135000, 79.279000, {"larva1": 50, "larva2": 30, "adult": 20}),
    (52.134500, 79.278500, {"larva3": 40, "larva4": 60}),
]

# ---------------------------
#  Гео-утилиты
# ---------------------------

def meters_to_deg_lat(m):
    return m / 111000.0

def meters_to_deg_lon(m, lat):
    return m / (111000.0 * math.cos(math.radians(lat)) + 1e-12)

def deg_lat_to_meters(dlat):
    return dlat * 111000.0

def deg_lon_to_meters(dlon, lat):
    return dlon * 111000.0 * math.cos(math.radians(lat))

def haversine_m(lat1, lon1, lat2, lon2):
    R = 6371000.0
    phi1 = math.radians(lat1); phi2 = math.radians(lat2)
    dphi = math.radians(lat2 - lat1); dlambda = math.radians(lon2 - lon1)
    a = math.sin(dphi/2)**2 + math.cos(phi1)*math.cos(phi2)*math.sin(dlambda/2)**2
    return 2*R*math.asin(math.sqrt(a))

# ---------------------------
#  Агент Podisus maculiventris
# ---------------------------

class PodisusIndividual:
    def __init__(self, lat, lon, stage, sex=None, dd_at_birth=0.0): # <- Новый параметр
        self.lat = float(lat)
        self.lon = float(lon)
        self.stage = stage
        self.alive = True
        self.dd_required = self.assign_dd_required(stage)
        self.dd_accum = dd_at_birth # <- Используем накопленное значение
        self.sex = sex if sex else random.choice(["F", "M"])
        self.age_days = 0
        self.last_mated = -100

    def assign_dd_required(self, stage):
        if stage == "egg":
            return EGG_DD_PODISUS
        if stage.startswith("nymph"):
            total = random.uniform(NYMPH_DD_RANGE_PODISUS[0], NYMPH_DD_RANGE_PODISUS[1])
            idx = int(stage.replace("nymph", "")) - 1
            return total * float(NYMPH_STAGE_PROP_PODISUS[idx])
        return float('inf')

    def progress(self, dd_since_year_start):
        if not self.alive or self.stage == "adult":
            return
        # Рассчитываем DD, накопленные с начала года до текущего дня
        current_dd = dd_since_year_start
        # Накопленный DD агента - это DD с начала года до его рождения + DD с его рождения до текущего дня
        # dd_accum - это DD с рождения до текущего дня
        # current_dd - это DD с начала года до текущего дня
        # Предполагаем, что dd_since_year_start_at_birth = current_dd - dd_accum
        # Тогда новый dd_accum = current_dd - dd_since_year_start_at_birth = current_dd - (current_dd_old - dd_accum_old)
        # Это упрощается до: dd_accum = current_dd - (dd_since_year_start_at_birth)
        # Но проще: dd_accum = current_dd - dd_since_year_start_at_birth
        # Изначально dd_since_year_start_at_birth = current_dd - dd_accum_initial
        # dd_accum = current_dd - (dd_since_year_start_at_birth)
        # Если dd_accum_initial = 0, то dd_since_year_start_at_birth = current_dd_now - 0 = current_dd_now
        # dd_accum = current_dd - current_dd_now (когда родился)
        # Это неправильно. Нужно хранить DD рождения.
        # Правильный подход:
        # У агента есть dd_accum - сколько он накопил *после* рождения.
        # У него есть dd_at_birth - сколько было накоплено к моменту рождения.
        # dd_since_year_start - общее накопление с начала года.
        # Тогда dd_accum_total = dd_at_birth + dd_accum_since_birth
        # dd_accum_since_birth = dd_since_year_start - dd_at_birth
        # dd_accum = max(0, dd_accum_since_birth) # На случай если dd_at_birth > dd_since_year_start
        dd_accum_since_birth = dd_since_year_start - self.dd_at_birth
        if dd_accum_since_birth < 0:
            # Это может быть, если агент родился в день с высокой температурой
            # и dd_at_birth был рассчитан как T_avg - T_base, а не накопленный DD
            # или если DD с начала года еще не дотянул до DD рождения.
            # Для простоты, если dd_at_birth > dd_since_year_start, считаем dd_accum = 0
            self.dd_accum = 0.0
        else:
            self.dd_accum = dd_accum_since_birth

        if self.dd_accum >= self.dd_required:
            if self.stage == "egg":
                self.stage = "nymph1"
            elif self.stage.startswith("nymph"):
                idx = int(self.stage.replace("nymph", ""))
                self.stage = f"nymph{idx+1}" if idx < 5 else "adult"
            self.dd_required = self.assign_dd_required(self.stage)
            # После перехода dd_accum сбрасывается, но это DD *после* перехода
            # Новый dd_at_birth для потомства будет dd_since_year_start
            self.dd_accum = 0.0

    def __init__(self, lat, lon, stage, sex=None, dd_at_birth=0.0): # Переопределяем __init__ с новым атрибутом
        self.lat = float(lat)
        self.lon = float(lon)
        self.stage = stage
        self.alive = True
        self.dd_required = self.assign_dd_required(stage)
        self.dd_at_birth = dd_at_birth # <- Новый атрибут: DD на момент рождения
        self.dd_accum = 0.0 # <- DD накопленный *после* рождения
        self.sex = sex if sex else random.choice(["F", "M"])
        self.age_days = 0
        self.last_mated = -100


# ---------------------------
#  Агент Leptinotarsa decemlineata
# ---------------------------

class LeptinotarsaIndividual:
    def __init__(self, lat, lon, stage, sex=None, dd_at_birth=0.0): # <- Новый параметр
        self.lat = float(lat)
        self.lon = float(lon)
        self.stage = stage
        self.alive = True
        self.dd_required = self.assign_dd_required(stage)
        self.dd_at_birth = dd_at_birth # <- Новый атрибут
        self.dd_accum = 0.0 # <- DD накопленный *после* рождения
        self.sex = sex if sex else random.choice(["F", "M"])
        self.age_days = 0
        self.last_mated = -100

    def assign_dd_required(self, stage):
        if stage == "egg":
            return DD_EGG_LEPTINO
        if stage.startswith("larva"):
            return DD_PER_LARVA_STAGE_LEPTINO
        if stage == "pupa":
            return DD_PUPA_LEPTINO
        return float('inf')

    def progress(self, dd_since_year_start):
        if not self.alive or self.stage == "adult":
            return
        dd_accum_since_birth = dd_since_year_start - self.dd_at_birth
        if dd_accum_since_birth < 0:
            self.dd_accum = 0.0
        else:
            self.dd_accum = dd_accum_since_birth

        if self.dd_accum >= self.dd_required:
            if self.stage == "egg":
                self.stage = "larva1"
            elif self.stage.startswith("larva"):
                idx = int(self.stage.replace("larva", ""))
                self.stage = f"larva{idx+1}" if idx < 4 else "pupa"
            elif self.stage == "pupa":
                self.stage = "adult"
            self.dd_required = self.assign_dd_required(self.stage)
            self.dd_accum = 0.0


# ---------------------------
#  Чтение метеоданных из архива (читаем с начала года)
# ---------------------------

def read_weather_archive(filepath: str, year_start_date: datetime.date, sim_start_date: datetime.date, sim_days: int):
    """
    Читает файл архива, начиная с year_start_date до (sim_start_date + sim_days - 1).
    Возвращает список словарей {'dt': date, 'temp': float, 'precip': float, 'dd': float}
    dd - накопленный Degree Day с начала года.
    """
    data = {}
    with open(filepath, 'r', encoding='utf-8') as f:
        for line in f:
            parts = line.strip().split(';')
            if len(parts) < 8:
                continue
            _, yr, mo, da, tmin, tavg, tmax, precip = parts
            try:
                yr = int(yr); mo = int(mo); da = int(da)
                date = datetime.date(yr, mo, da)
                data[date] = {
                    "tmin": float(tmin),
                    "tavg": float(tavg),
                    "tmax": float(tmax),
                    "precip": float(precip)
                }
            except:
                continue

    # Сортируем даты
    sorted_dates = sorted(data.keys())
    # Находим индекс начала и конца периода
    start_idx = 0
    end_idx = len(sorted_dates) - 1
    for i, d in enumerate(sorted_dates):
        if d == year_start_date:
            start_idx = i
        if d == sim_start_date + datetime.timedelta(days=sim_days - 1):
            end_idx = i
            break

    # Выбираем только нужный диапазон
    relevant_dates = sorted_dates[start_idx:end_idx+1]
    weather_list = []
    cumulative_dd = 0.0
    for date in relevant_dates:
        rec = data.get(date, {"tavg": 20.0, "precip": 0.0}) # Значения по умолчанию
        daily_dd = max(0.0, rec["tavg"] - 10.0) # Пример: T_base = 10.0 для общего накопления
        cumulative_dd += daily_dd
        weather_list.append({
            "dt": date,
            "temp": rec["tavg"],
            "precip": rec["precip"],
            "dd_cumulative": cumulative_dd
        })
    return weather_list, start_idx, len(relevant_dates) - 1 # Возвращаем также индексы для удобства

# ---------------------------
#  Симулятор (обновлен для учета DD с начала года)
# ---------------------------

class CombinedSimulator:
    def __init__(self,
                 init_points: List[Tuple[float,float]],
                 init_counts_podisus: dict,
                 init_counts_leptinotarsa: dict,
                 sim_days: int,
                 weather_filepath: str,
                 weather_start_date_str: str, # Принимаем строку
                 manual_prey_points: List[Tuple[float, float, Dict[str, int]]] = None,
                 year_start_date: datetime.date = YEAR_START_DATE, # Новый параметр
                 geojson_path: Optional[str] = None):
        random.seed(42)
        np.random.seed(42)
        self.sim_days = sim_days
        self.agents_podisus: List[PodisusIndividual] = []
        self.agents_leptinotarsa: List[LeptinotarsaIndividual] = []
        self.init_points = init_points

        # Парсим дату начала симуляции
        self.sim_start_date = datetime.datetime.strptime(weather_start_date_str, "%d.%m.%Y").date()

        # Читаем погоду с начала года до конца симуляции
        self.weather, self.start_day_index, self.end_day_index = read_weather_archive(
            weather_filepath, year_start_date, self.sim_start_date, sim_days
        )

        # Инициализация Podisus
        for lat, lon in init_points:
            for stg, cnt in init_counts_podisus.items():
                for _ in range(cnt):
                    # Предполагаем, что агенты созданы в день начала симуляции
                    dd_at_birth = self.weather[self.start_day_index]["dd_cumulative"] # DD на момент начала симуляции
                    self.agents_podisus.append(PodisusIndividual(lat, lon, stg, dd_at_birth=dd_at_birth))

        # Инициализация Leptinotarsa из начальной точки
        for lat, lon in init_points:
            for stg, cnt in init_counts_leptinotarsa.items():
                for _ in range(cnt):
                    dd_at_birth = self.weather[self.start_day_index]["dd_cumulative"]
                    self.agents_leptinotarsa.append(LeptinotarsaIndividual(lat, lon, stg, dd_at_birth=dd_at_birth))

        # Инициализация Leptinotarsa вручную заданных точек
        if manual_prey_points:
            for lat, lon, counts_dict in manual_prey_points:
                for stg, cnt in counts_dict.items():
                    for _ in range(cnt):
                        lat_offset = meters_to_deg_lat(random.uniform(-2, 2))
                        lon_offset = meters_to_deg_lon(random.uniform(-2, 2), lat)
                        dd_at_birth = self.weather[self.start_day_index]["dd_cumulative"]
                        self.agents_leptinotarsa.append(LeptinotarsaIndividual(lat + lat_offset, lon + lon_offset, stg, dd_at_birth=dd_at_birth))

        # Границы поля (опционально)
        self.boundary_polygon: Optional[Polygon] = None
        if geojson_path and os.path.exists(geojson_path):
            with open(geojson_path, 'r', encoding='utf-8') as f:
                gj = json.load(f)
            geom = None
            if gj.get("type") == "FeatureCollection":
                polys = [shape(feat["geometry"]) for feat in gj["features"]]
                geom = polys[0].union(*polys[1:]) if len(polys) > 1 else polys[0]
            elif gj.get("type") == "Feature":
                geom = shape(gj["geometry"])
            else:
                geom = shape(gj)
            self.boundary_polygon = geom

        self.history_podisus = []
        self.history_leptinotarsa = []
        os.makedirs("simulation_output", exist_ok=True)

    def run(self):
        for day in tqdm(range(self.sim_days), desc="Simulating days"):
            # Получаем погоду для текущего дня симуляции (day=0 -> sim_start_date)
            # Индекс в общем массиве weather
            current_weather_index = self.start_day_index + day
            if current_weather_index >= len(self.weather):
                break # Защита от выхода за пределы, если погода закончилась
            w = self.weather[current_weather_index]
            T = w["temp"]
            precip = w.get("precip", 0.0)
            dd_since_year_start = w["dd_cumulative"] # <- Используем накопленный DD

            # --- Обновление Podisus ---
            alive_podisus = [a for a in self.agents_podisus if a.alive]
            for a in alive_podisus:
                a.age_days += 1
                # --- Смертность ---
                mort = BASE_MORT_PODISUS.get(a.stage, 0.01)
                if T > UPPER_TEMP_THRESHOLD_PODISUS or T < LOWER_TEMP_THRESHOLD_PODISUS:
                    mort *= 2.0
                if a.stage.startswith("nymph"):
                    nearby = [x for x in alive_podisus if x != a and x.alive and haversine_m(a.lat, a.lon, x.lat, x.lon) <= DENSITY_RADIUS]
                    if len(nearby) >= DENSITY_THRESHOLD:
                        mort += len(nearby) * DENSITY_MORTALITY_FACTOR
                if random.random() < mort:
                    a.alive = False
                    continue

                # --- Развитие ---
                # dd_since_year_start передается в progress
                a.progress(dd_since_year_start)

                # --- Размножение (только после начала симуляции) ---
                if a.stage == "adult" and a.sex == "F" and a.age_days >= MIN_AGE_ADULT_PODISUS:
                    if random.random() < 0.1: # 10% шанс на размножение в день
                        for _ in range(10):
                            new_lat = a.lat + meters_to_deg_lat(random.uniform(-1, 1))
                            new_lon = a.lon + meters_to_deg_lon(random.uniform(-1, 1), a.lat)
                            # Новое потомство рождается в текущий день симуляции
                            dd_at_birth_new = dd_since_year_start
                            self.agents_podisus.append(PodisusIndividual(new_lat, new_lon, "egg", dd_at_birth=dd_at_birth_new))

                # --- Движение ---
                step_m = MOVE_BASE_METERS_PODISUS.get(a.stage, 0.0)
                if step_m > 0:
                    min_d = float('inf')
                    target = None
                    for prey_agent in self.agents_leptinotarsa:
                        if prey_agent.alive:
                            d = haversine_m(a.lat, a.lon, prey_agent.lat, prey_agent.lon)
                            if d < min_d:
                                min_d = d
                                target = (prey_agent.lat, prey_agent.lon)

                    if target and min_d <= PREY_DETECTION_RADIUS_PODISUS:
                        vy = deg_lat_to_meters(target[0] - a.lat)
                        vx = deg_lon_to_meters(target[1] - a.lon, a.lat)
                        norm = math.hypot(vx, vy) + 1e-12
                        ux, uy = vx/norm, vy/norm
                        theta = random.random() * 2*math.pi
                        rx, ry = math.cos(theta), math.sin(theta)
                        vx_m = PREY_BIAS_PODISUS * ux + (1 - PREY_BIAS_PODISUS) * rx
                        vy_m = PREY_BIAS_PODISUS * uy + (1 - PREY_BIAS_PODISUS) * ry
                        norm2 = math.hypot(vx_m, vy_m) + 1e-12
                        vx_m = vx_m / norm2 * step_m
                        vy_m = vy_m / norm2 * step_m
                    else:
                        theta = random.random() * 2*math.pi
                        vx_m = step_m * math.cos(theta)
                        vy_m = step_m * math.sin(theta)

                    dlat = meters_to_deg_lat(vy_m)
                    dlon = meters_to_deg_lon(vx_m, a.lat)
                    new_lat = a.lat + dlat
                    new_lon = a.lon + dlon

                    if self.boundary_polygon is not None:
                        pt = Point(new_lon, new_lat)
                        if not self.boundary_polygon.contains(pt):
                            new_lat, new_lon = a.lat, a.lon

                    a.lat, a.lon = new_lat, new_lon

            # --- Обновление Leptinotarsa ---
            alive_leptino = [a for a in self.agents_leptinotarsa if a.alive]
            for a in alive_leptino:
                a.age_days += 1
                # --- Смертность ---
                mort = BASE_MORT_LEPTINO.get(a.stage, 0.01)
                if T > UPPER_TEMP_THRESHOLD_LEPTINO or T < LOWER_TEMP_THRESHOLD_LEPTINO:
                    mort *= 2.0
                if a.stage.startswith("larva"):
                    nearby = [x for x in alive_leptino if x != a and x.alive and haversine_m(a.lat, a.lon, x.lat, x.lon) <= DENSITY_RADIUS]
                    if len(nearby) >= DENSITY_THRESHOLD:
                        mort += len(nearby) * DENSITY_MORTALITY_FACTOR
                if a.stage.startswith("larva"):
                    nearby_predators = [p for p in alive_podisus if haversine_m(a.lat, a.lon, p.lat, p.lon) <= PREY_DETECTION_RADIUS_PODISUS]
                    if nearby_predators:
                        if random.random() < 0.5:
                            a.alive = False
                            continue
                if random.random() < mort:
                    a.alive = False
                    continue

                # --- Развитие ---
                a.progress(dd_since_year_start)

                # --- Размножение (только после начала симуляции) ---
                if a.stage == "adult" and a.sex == "F" and a.age_days >= MIN_AGE_ADULT_LEPTINO:
                    if random.random() < 0.05: # 5% шанс на размножение в день
                        for _ in range(CLUTCH_SIZE_LEPTINO):
                            new_lat = a.lat + meters_to_deg_lat(random.uniform(-1, 1))
                            new_lon = a.lon + meters_to_deg_lon(random.uniform(-1, 1), a.lat)
                            # Новое потомство рождается в текущий день симуляции
                            dd_at_birth_new = dd_since_year_start
                            self.agents_leptinotarsa.append(LeptinotarsaIndividual(new_lat, new_lon, "egg", dd_at_birth=dd_at_birth_new))

                # --- Движение ---
                step_m = MOVE_BASE_METERS_LEPTINO.get(a.stage, 0.0)
                if step_m > 0:
                    theta = random.random() * 2*math.pi
                    vx_m = step_m * math.cos(theta)
                    vy_m = step_m * math.sin(theta)

                    dlat = meters_to_deg_lat(vy_m)
                    dlon = meters_to_deg_lon(vx_m, a.lat)
                    new_lat = a.lat + dlat
                    new_lon = a.lon + dlon

                    if self.boundary_polygon is not None:
                        pt = Point(new_lon, new_lat)
                        if not self.boundary_polygon.contains(pt):
                            new_lat, new_lon = a.lat, a.lon

                    a.lat, a.lon = new_lat, new_lon

            # Сохранение истории
            day_points_podisus = [(a.lat, a.lon, a.stage) for a in self.agents_podisus if a.alive]
            day_points_leptino = [(a.lat, a.lon, a.stage) for a in self.agents_leptinotarsa if a.alive]

            self.history_podisus.append(day_points_podisus)
            self.history_leptinotarsa.append(day_points_leptino)

            self.export_geojson_day(day, day_points_podisus, "podisus")
            self.export_geojson_day(day, day_points_leptino, "leptinotarsa")
            self.export_geotiff_day(day, day_points_podisus, "podisus")
            self.export_geotiff_day(day, day_points_leptino, "leptinotarsa")

    def export_geojson_day(self, d, pts, species_name):
        feats = []
        for (lat, lon, stg) in pts:
            feats.append({
                "type":"Feature",
                "geometry":{"type":"Point","coordinates":[lon, lat]},
                "properties":{"stage":stg, "species": species_name}
            })
        fc = {"type":"FeatureCollection", "features":feats}
        filename = f"simulation_output/{species_name}_points_day_{d:03d}.geojson"
        with open(filename, "w", encoding="utf-8") as f:
            json.dump(fc, f, ensure_ascii=False, indent=2)

    def export_geotiff_day(self, d, pts, species_name):
        if not pts:
            return
        lats = [p[0] for p in pts]; lons = [p[1] for p in pts]
        min_lat, max_lat = min(lats), max(lats); min_lon, max_lon = min(lons), max(lons)
        pad = 20.0
        padlat = meters_to_deg_lat(pad); padlon = meters_to_deg_lon(pad, (min_lat+max_lat)/2.0)
        min_lat -= padlat; max_lat += padlat; min_lon -= padlon; max_lon += padlon

        center_lat = (min_lat + max_lat)/2.0
        x_min_m = deg_lon_to_meters(min_lon, center_lat)
        x_max_m = deg_lon_to_meters(max_lon, center_lat)
        y_min_m = deg_lat_to_meters(min_lat)
        y_max_m = deg_lat_to_meters(max_lat)
        width_m = x_max_m - x_min_m; height_m = y_max_m - y_min_m
        if width_m <= 0 or height_m <= 0:
            return
        px = max(1, int(math.ceil(width_m / PIXEL_SIZE_M)))
        py = max(1, int(math.ceil(height_m / PIXEL_SIZE_M)))

        pix_h = meters_to_deg_lat(PIXEL_SIZE_M); pix_w = meters_to_deg_lon(PIXEL_SIZE_M, center_lat)
        transform = from_origin(min_lon, max_lat, pix_w, pix_h)

        raster = np.zeros((py, px), dtype=np.uint16)
        for lat, lon, _ in pts:
            col = int((lon - min_lon) / pix_w)
            row = int((max_lat - lat) / pix_h)
            if 0 <= row < py and 0 <= col < px:
                raster[row, col] += 1

        filename = f"simulation_output/{species_name}_density_day_{d:03d}.tif"
        with rasterio.open(filename, 'w',
                           driver='GTiff', height=py, width=px,
                           count=1, dtype=raster.dtype,
                           crs="EPSG:4326", transform=transform, compress='lzw') as dst:
            dst.write(raster, 1)




In [4]:
# ---------------------------
#  Параметры симуляции
# ---------------------------

# Инициализация начальных точек и количества особей (Podisus)
INIT_POINTS = [(52.134873, 79.278759)]
INIT_COUNTS_PODISUS = {"egg": 0, "nymph1": 0, "nymph2": 0, "nymph3": 0, "nymph4": 0, "nymph5": 0, "adult": 1000}
INIT_COUNTS_LEPTINOTARSA = {"egg": 0, "larva1": 0, "larva2": 0, "larva3": 0, "larva4": 0, "pupa": 0, "adult": 500}

SIM_DAYS = 30

# --- Параметры Podisus maculiventris (хищный клоп) ---
# DD (Degree Days)
NYMPH_DD_RANGE_PODISUS = (233.9, 338.7)
EGG_DD_PODISUS = 50.0
BASE_TEMP_EGG_PODISUS = 10.7
BASE_TEMP_NYMPH_PODISUS = 11.7
NYMPH_STAGE_PROP_PODISUS = np.array([0.10, 0.18, 0.20, 0.25, 0.27])
BASE_MORT_PODISUS = {
    "egg": 0.01, "nymph1": 0.01, "nymph2": 0.007, "nymph3": 0.005, "nymph4": 0.005, "nymph5": 0.005, "adult": 0.002
}
MOVE_BASE_METERS_PODISUS = {
    "egg": 0.0, "nymph1": 0.5, "nymph2": 2.0, "nymph3": 6.0, "nymph4": 10.0, "nymph5": 15.0, "adult": 25.0
}
PREY_DETECTION_RADIUS_PODISUS = 10.0
PREY_BIAS_PODISUS = 0.8
UPPER_TEMP_THRESHOLD_PODISUS = 35.0
LOWER_TEMP_THRESHOLD_PODISUS = 10.0
MIN_AGE_ADULT_PODISUS = 5

# --- Параметры Leptinotarsa decemlineata (колорадский жук) ---
BASE_TEMP_EGG_LEPTINO = 11.5
BASE_TEMP_LARVA_LEPTINO = 11.5
BASE_TEMP_PUPA_LEPTINO = 11.5
BASE_TEMP_ADULT_LEPTINO = 11.5
DD_EGG_LEPTINO = 40.0
DD_LARVA_TOTAL_LEPTINO = 150.0
DD_PUPA_LEPTINO = 100.0
DD_PER_LARVA_STAGE_LEPTINO = DD_LARVA_TOTAL_LEPTINO / 4
BASE_MORT_LEPTINO = {
    "egg": 0.05, "larva1": 0.1, "larva2": 0.1, "larva3": 0.1, "larva4": 0.1, "pupa": 0.05, "adult": 0.005
}
MOVE_BASE_METERS_LEPTINO = {
    "egg": 0.0, "larva1": 0.1, "larva2": 0.2, "larva3": 0.5, "larva4": 1.0, "pupa": 0.0, "adult": 10.0
}
CONSUMPTION_LARVA_LEPTINO = [0.24, 0.50, 2.04, 4.15] # см2/день по стадиям 1-4
CONSUMPTION_ADULT_LEPTINO = 10.0 # см2/день
MIN_AGE_ADULT_LEPTINO = 5
CLUTCH_SIZE_LEPTINO = 30
FECUNDITY_LEPTINO = 10
UPPER_TEMP_THRESHOLD_LEPTINO = 33.0
LOWER_TEMP_THRESHOLD_LEPTINO = 10.0

# --- Общие параметры ---
PIXEL_SIZE_M = 1.0
DENSITY_RADIUS = math.sqrt(0.1 / math.pi)
DENSITY_THRESHOLD = 3
DENSITY_MORTALITY_FACTOR = 0.1

# --- НОВОЕ: Дата начала года для DD ---
YEAR_START_DATE = datetime.date(2024, 1, 1) # <- Задается вручную
MANUAL_PREY_POINTS = [
    (52.135000, 79.279000, {"larva1": 50, "larva2": 30, "adult": 20}),
    (52.134500, 79.278500, {"larva3": 40, "larva4": 60}),
]

# ---------------------------
#  Основной запуск
# ---------------------------

if __name__ == "__main__":
    weather_file = r"C:\Users\pochy\Desktop\Стартап Экологические услуги\Модель расселения\wr352396a1\wr352396a1.txt"
    sim_start_date_str = "15.06.2024"
    sim = CombinedSimulator(
        init_points = INIT_POINTS,
        init_counts_podisus = INIT_COUNTS_PODISUS,
        init_counts_leptinotarsa = INIT_COUNTS_LEPTINOTARSA,
        sim_days = SIM_DAYS,
        weather_filepath = weather_file,
        weather_start_date_str = sim_start_date_str, # Передаем строку
        manual_prey_points=MANUAL_PREY_POINTS,
        year_start_date=YEAR_START_DATE, # Передаем дату начала года
        geojson_path = None
    )
    sim.run()
    print("Done. Outputs in simulation_output/")

Simulating days: 100%|██████████| 30/30 [01:16<00:00,  2.56s/it]

Done. Outputs in simulation_output/


Попытки учитывать границы

In [5]:
import math
import random
import datetime
import json
import os
from typing import List, Tuple, Optional, Dict, Any
import numpy as np
from shapely.geometry import Point, shape, Polygon, LineString
import rasterio
from rasterio.transform import from_origin
from tqdm import tqdm

# ---------------------------
#  Параметры симуляции
# ---------------------------

# Инициализация начальных точек и количества особей (Podisus)
INIT_POINTS = [(52.134873, 79.278759)]
INIT_COUNTS_PODISUS = {"egg": 0, "nymph1": 0, "nymph2": 0, "nymph3": 1000, "nymph4": 0, "nymph5": 0, "adult": 0}
INIT_COUNTS_LEPTINOTARSA = {"egg": 0, "larva1": 0, "larva2": 0, "larva3": 0, "larva4": 2000, "pupa": 0, "adult": 0}

SIM_DAYS = 30

# --- Параметры Podisus maculiventris (хищный клоп) ---
# DD (Degree Days)
NYMPH_DD_RANGE_PODISUS = (233.9, 338.7)
EGG_DD_PODISUS = 50.0
BASE_TEMP_EGG_PODISUS = 10.7
BASE_TEMP_NYMPH_PODISUS = 11.7
NYMPH_STAGE_PROP_PODISUS = np.array([0.10, 0.18, 0.20, 0.25, 0.27])
BASE_MORT_PODISUS = {
    "egg": 0.01, "nymph1": 0.01, "nymph2": 0.007, "nymph3": 0.005, "nymph4": 0.005, "nymph5": 0.005, "adult": 0.002
}
MOVE_BASE_METERS_PODISUS = {
    "egg": 0.0, "nymph1": 0.5, "nymph2": 2.0, "nymph3": 6.0, "nymph4": 10.0, "nymph5": 15.0, "adult": 25.0
}
PREY_DETECTION_RADIUS_PODISUS = 10.0
PREY_BIAS_PODISUS = 0.8 # <- Уменьшено для большей хаотичности
UPPER_TEMP_THRESHOLD_PODISUS = 35.0
LOWER_TEMP_THRESHOLD_PODISUS = 10.0
MIN_AGE_ADULT_PODISUS = 5

# --- Параметры Leptinotarsa decemlineata (колорадский жук) ---
BASE_TEMP_EGG_LEPTINO = 11.5
BASE_TEMP_LARVA_LEPTINO = 11.5
BASE_TEMP_PUPA_LEPTINO = 11.5
BASE_TEMP_ADULT_LEPTINO = 11.5
DD_EGG_LEPTINO = 40.0
DD_LARVA_TOTAL_LEPTINO = 150.0
DD_PUPA_LEPTINO = 100.0
DD_PER_LARVA_STAGE_LEPTINO = DD_LARVA_TOTAL_LEPTINO / 4
BASE_MORT_LEPTINO = {
    "egg": 0.05, "larva1": 0.1, "larva2": 0.1, "larva3": 0.1, "larva4": 0.1, "pupa": 0.05, "adult": 0.005
}
MOVE_BASE_METERS_LEPTINO = {
    "egg": 0.0, "larva1": 0.1, "larva2": 0.2, "larva3": 0.5, "larva4": 1.0, "pupa": 0.0, "adult": 10.0
}
CONSUMPTION_LARVA_LEPTINO = [0.24, 0.50, 2.04, 4.15] # см2/день по стадиям 1-4
CONSUMPTION_ADULT_LEPTINO = 10.0 # см2/день
MIN_AGE_ADULT_LEPTINO = 5
CLUTCH_SIZE_LEPTINO = 30
FECUNDITY_LEPTINO = 10
UPPER_TEMP_THRESHOLD_LEPTINO = 33.0
LOWER_TEMP_THRESHOLD_LEPTINO = 10.0

# --- Общие параметры ---
PIXEL_SIZE_M = 1.0
DENSITY_RADIUS = math.sqrt(0.1 / math.pi)
DENSITY_THRESHOLD = 3
DENSITY_MORTALITY_FACTOR = 0.1

# --- НОВОЕ: Дата начала года для DD ---
YEAR_START_DATE = datetime.date(2024, 1, 1) # <- Задается вручную
MANUAL_PREY_POINTS = [
    (52.135000, 79.279000, {"larva1": 50, "larva2": 30, "adult": 20}),
    (52.134500, 79.278500, {"larva3": 40, "larva4": 60}),
]

# --- Путь к файлу с преградами ---
BORDER_GEOJSON_PATH = r"C:\Users\pochy\Desktop\Стартап Экологические услуги\Модель расселения\border.geojson"

# ---------------------------
#  Гео-утилиты
# ---------------------------

def meters_to_deg_lat(m):
    return m / 111000.0

def meters_to_deg_lon(m, lat):
    return m / (111000.0 * math.cos(math.radians(lat)) + 1e-12)

def deg_lat_to_meters(dlat):
    return dlat * 111000.0

def deg_lon_to_meters(dlon, lat):
    return dlon * 111000.0 * math.cos(math.radians(lat))

def haversine_m(lat1, lon1, lat2, lon2):
    R = 6371000.0
    phi1 = math.radians(lat1); phi2 = math.radians(lat2)
    dphi = math.radians(lat2 - lat1); dlambda = math.radians(lon2 - lon1)
    a = math.sin(dphi/2)**2 + math.cos(phi1)*math.cos(phi2)*math.sin(dlambda/2)**2
    return 2*R*math.asin(math.sqrt(a))

# ---------------------------
#  Загрузка и обработка преград
# ---------------------------

def load_obstacles(geojson_path: str):
    """
    Загружает геометрии преград из GeoJSON файла.
    Возвращает список словарей {'geometry': Polygon, 'grad': int}.
    """
    obstacles = []
    if os.path.exists(geojson_path):
        with open(geojson_path, 'r', encoding='utf-8') as f:
            gj = json.load(f)
        features = gj.get("features", [])
        for feat in features:
            geom = shape(feat.get("geometry"))
            # Убедимся, что геометрия является Polygon или MultiPolygon
            if geom.geom_type == 'MultiPolygon':
                # Разбиваем MultiPolygon на отдельные Polygon
                for poly in geom.geoms:
                    grad = feat.get("properties", {}).get("grad", 8)
                    if grad is None:
                        grad = 8
                    obstacles.append({'geometry': poly, 'grad': grad})
            elif geom.geom_type == 'Polygon':
                grad = feat.get("properties", {}).get("grad", 8)
                if grad is None:
                    grad = 8
                obstacles.append({'geometry': geom, 'grad': grad})
    return obstacles

def get_move_probability(obstacles, from_point, to_point):
    """
    Определяет вероятность перемещения от from_point к to_point.
    Проверяет пересечение с преградами.
    grad = 0 -> вероятность 0.
    grad > 0 -> вероятность 1 (для простоты, можно сделать более сложной).
    grad = None или отсутствует -> считаем как 8.
    Если точка не входит в преграду, считаем grad=8.
    """
    line = LineString([from_point, to_point])
    for obs in obstacles:
        if obs['geometry'].intersects(line):
            grad = obs['grad']
            if grad == 0:
                return 0.0 # Невозможно пройти
            # Для других grad > 0, принимаем вероятность = 1.0
            # (Можно изменить логику, например, prob = grad / max_grad)
    return 1.0 # Нет пересечений с преградами grad=0

# ---------------------------
#  Агент Podisus maculiventris
# ---------------------------

class PodisusIndividual:
    def __init__(self, lat, lon, stage, sex=None, dd_at_birth=0.0):
        self.lat = float(lat)
        self.lon = float(lon)
        self.stage = stage
        self.alive = True
        self.dd_required = self.assign_dd_required(stage)
        self.dd_at_birth = dd_at_birth
        self.dd_accum = 0.0
        self.sex = sex if sex else random.choice(["F", "M"])
        self.age_days = 0
        self.last_mated = -100

    def assign_dd_required(self, stage):
        if stage == "egg":
            return EGG_DD_PODISUS
        if stage.startswith("nymph"):
            total = random.uniform(NYMPH_DD_RANGE_PODISUS[0], NYMPH_DD_RANGE_PODISUS[1])
            idx = int(stage.replace("nymph", "")) - 1
            return total * float(NYMPH_STAGE_PROP_PODISUS[idx])
        return float('inf')

    def progress(self, dd_since_year_start):
        if not self.alive or self.stage == "adult":
            return
        dd_accum_since_birth = dd_since_year_start - self.dd_at_birth
        if dd_accum_since_birth < 0:
            self.dd_accum = 0.0
        else:
            self.dd_accum = dd_accum_since_birth

        if self.dd_accum >= self.dd_required:
            if self.stage == "egg":
                self.stage = "nymph1"
            elif self.stage.startswith("nymph"):
                idx = int(self.stage.replace("nymph", ""))
                self.stage = f"nymph{idx+1}" if idx < 5 else "adult"
            self.dd_required = self.assign_dd_required(self.stage)
            self.dd_accum = 0.0


# ---------------------------
#  Агент Leptinotarsa decemlineata
# ---------------------------

class LeptinotarsaIndividual:
    def __init__(self, lat, lon, stage, sex=None, dd_at_birth=0.0):
        self.lat = float(lat)
        self.lon = float(lon)
        self.stage = stage
        self.alive = True
        self.dd_required = self.assign_dd_required(stage)
        self.dd_at_birth = dd_at_birth
        self.dd_accum = 0.0
        self.sex = sex if sex else random.choice(["F", "M"])
        self.age_days = 0
        self.last_mated = -100

    def assign_dd_required(self, stage):
        if stage == "egg":
            return DD_EGG_LEPTINO
        if stage.startswith("larva"):
            return DD_PER_LARVA_STAGE_LEPTINO
        if stage == "pupa":
            return DD_PUPA_LEPTINO
        return float('inf')

    def progress(self, dd_since_year_start):
        if not self.alive or self.stage == "adult":
            return
        dd_accum_since_birth = dd_since_year_start - self.dd_at_birth
        if dd_accum_since_birth < 0:
            self.dd_accum = 0.0
        else:
            self.dd_accum = dd_accum_since_birth

        if self.dd_accum >= self.dd_required:
            if self.stage == "egg":
                self.stage = "larva1"
            elif self.stage.startswith("larva"):
                idx = int(self.stage.replace("larva", ""))
                self.stage = f"larva{idx+1}" if idx < 4 else "pupa"
            elif self.stage == "pupa":
                self.stage = "adult"
            self.dd_required = self.assign_dd_required(self.stage)
            self.dd_accum = 0.0


# ---------------------------
#  Чтение метеоданных из архива (читаем с начала года)
# ---------------------------

def read_weather_archive(filepath: str, year_start_date: datetime.date, sim_start_date: datetime.date, sim_days: int):
    """
    Читает файл архива, начиная с year_start_date до (sim_start_date + sim_days - 1).
    Возвращает список словарей {'dt': date, 'temp': float, 'precip': float, 'dd': float}
    dd - накопленный Degree Day с начала года.
    """
    data = {}
    with open(filepath, 'r', encoding='utf-8') as f:
        for line in f:
            parts = line.strip().split(';')
            if len(parts) < 8:
                continue
            _, yr, mo, da, tmin, tavg, tmax, precip = parts
            try:
                yr = int(yr); mo = int(mo); da = int(da)
                date = datetime.date(yr, mo, da)
                data[date] = {
                    "tmin": float(tmin),
                    "tavg": float(tavg),
                    "tmax": float(precip),
                    "precip": float(precip)
                }
            except:
                continue

    # Сортируем даты
    sorted_dates = sorted(data.keys())
    # Находим индекс начала и конца периода
    start_idx = 0
    end_idx = len(sorted_dates) - 1
    for i, d in enumerate(sorted_dates):
        if d == year_start_date:
            start_idx = i
        if d == sim_start_date + datetime.timedelta(days=sim_days - 1):
            end_idx = i
            break

    # Выбираем только нужный диапазон
    relevant_dates = sorted_dates[start_idx:end_idx+1]
    weather_list = []
    cumulative_dd = 0.0
    for date in relevant_dates:
        rec = data.get(date, {"tavg": 20.0, "precip": 0.0}) # Значения по умолчанию
        daily_dd = max(0.0, rec["tavg"] - 10.0) # Пример: T_base = 10.0 для общего накопления
        cumulative_dd += daily_dd
        weather_list.append({
            "dt": date,
            "temp": rec["tavg"],
            "precip": rec["precip"],
            "dd_cumulative": cumulative_dd
        })
    return weather_list, start_idx, len(relevant_dates) - 1


# ---------------------------
#  Симулятор (обновлен для учета DD с начала года и преград)
# ---------------------------

class CombinedSimulator:
    def __init__(self,
                 init_points: List[Tuple[float,float]],
                 init_counts_podisus: dict,
                 init_counts_leptinotarsa: dict,
                 sim_days: int,
                 weather_filepath: str,
                 weather_start_date_str: str,
                 manual_prey_points: List[Tuple[float, float, Dict[str, int]]] = None,
                 year_start_date: datetime.date = YEAR_START_DATE,
                 border_geojson_path: str = BORDER_GEOJSON_PATH, # Новый параметр
                 geojson_path: Optional[str] = None):
        random.seed(42)
        np.random.seed(42)
        self.sim_days = sim_days
        self.agents_podisus: List[PodisusIndividual] = []
        self.agents_leptinotarsa: List[LeptinotarsaIndividual] = []
        self.init_points = init_points

        # Парсим дату начала симуляции
        self.sim_start_date = datetime.datetime.strptime(weather_start_date_str, "%d.%m.%Y").date()

        # Читаем погоду с начала года до конца симуляции
        self.weather, self.start_day_index, self.end_day_index = read_weather_archive(
            weather_filepath, year_start_date, self.sim_start_date, sim_days
        )

        # Загружаем преграды
        self.obstacles = load_obstacles(border_geojson_path)

        # Инициализация Podisus
        for lat, lon in init_points:
            for stg, cnt in init_counts_podisus.items():
                for _ in range(cnt):
                    dd_at_birth = self.weather[self.start_day_index]["dd_cumulative"]
                    self.agents_podisus.append(PodisusIndividual(lat, lon, stg, dd_at_birth=dd_at_birth))

        # Инициализация Leptinotarsa из начальной точки
        for lat, lon in init_points:
            for stg, cnt in init_counts_leptinotarsa.items():
                for _ in range(cnt):
                    dd_at_birth = self.weather[self.start_day_index]["dd_cumulative"]
                    self.agents_leptinotarsa.append(LeptinotarsaIndividual(lat, lon, stg, dd_at_birth=dd_at_birth))

        # Инициализация Leptinotarsa вручную заданных точек
        if manual_prey_points:
            for lat, lon, counts_dict in manual_prey_points:
                for stg, cnt in counts_dict.items():
                    for _ in range(cnt):
                        lat_offset = meters_to_deg_lat(random.uniform(-2, 2))
                        lon_offset = meters_to_deg_lon(random.uniform(-2, 2), lat)
                        dd_at_birth = self.weather[self.start_day_index]["dd_cumulative"]
                        self.agents_leptinotarsa.append(LeptinotarsaIndividual(lat + lat_offset, lon + lon_offset, stg, dd_at_birth=dd_at_birth))

        # Границы поля (опционально)
        self.boundary_polygon: Optional[Polygon] = None
        if geojson_path and os.path.exists(geojson_path):
            with open(geojson_path, 'r', encoding='utf-8') as f:
                gj = json.load(f)
            geom = None
            if gj.get("type") == "FeatureCollection":
                polys = [shape(feat["geometry"]) for feat in gj["features"]]
                geom = polys[0].union(*polys[1:]) if len(polys) > 1 else polys[0]
            elif gj.get("type") == "Feature":
                geom = shape(gj["geometry"])
            else:
                geom = shape(gj)
            self.boundary_polygon = geom

        self.history_podisus = []
        self.history_leptinotarsa = []
        os.makedirs("simulation_output", exist_ok=True)

    def run(self):
        for day in tqdm(range(self.sim_days), desc="Simulating days"):
            # Получаем погоду для текущего дня симуляции
            current_weather_index = self.start_day_index + day
            if current_weather_index >= len(self.weather):
                break
            w = self.weather[current_weather_index]
            T = w["temp"]
            precip = w.get("precip", 0.0)
            dd_since_year_start = w["dd_cumulative"]

            # --- Обновление Podisus ---
            alive_podisus = [a for a in self.agents_podisus if a.alive]
            for a in alive_podisus:
                a.age_days += 1
                # --- Смертность ---
                mort = BASE_MORT_PODISUS.get(a.stage, 0.01)
                if T > UPPER_TEMP_THRESHOLD_PODISUS or T < LOWER_TEMP_THRESHOLD_PODISUS:
                    mort *= 2.0
                if a.stage.startswith("nymph"):
                    nearby = [x for x in alive_podisus if x != a and x.alive and haversine_m(a.lat, a.lon, x.lat, x.lon) <= DENSITY_RADIUS]
                    if len(nearby) >= DENSITY_THRESHOLD:
                        mort += len(nearby) * DENSITY_MORTALITY_FACTOR
                if random.random() < mort:
                    a.alive = False
                    continue

                # --- Развитие ---
                a.progress(dd_since_year_start)

                # --- Размножение (только после начала симуляции) ---
                if a.stage == "adult" and a.sex == "F" and a.age_days >= MIN_AGE_ADULT_PODISUS:
                    if random.random() < 0.1:
                        for _ in range(10):
                            new_lat = a.lat + meters_to_deg_lat(random.uniform(-1, 1))
                            new_lon = a.lon + meters_to_deg_lon(random.uniform(-1, 1), a.lat)
                            dd_at_birth_new = dd_since_year_start
                            self.agents_podisus.append(PodisusIndividual(new_lat, new_lon, "egg", dd_at_birth=dd_at_birth_new))

                # --- Движение ---
                step_m = MOVE_BASE_METERS_PODISUS.get(a.stage, 0.0)
                if step_m > 0:
                    min_d = float('inf')
                    target = None
                    for prey_agent in self.agents_leptinotarsa:
                        if prey_agent.alive:
                            d = haversine_m(a.lat, a.lon, prey_agent.lat, prey_agent.lon)
                            if d < min_d:
                                min_d = d
                                target = (prey_agent.lat, prey_agent.lon)

                    # --- НОВОЕ: Логика движения с учетом преград и хаоса ---
                    if target and min_d <= PREY_DETECTION_RADIUS_PODISUS:
                        # Направление к добыче
                        vy = deg_lat_to_meters(target[0] - a.lat)
                        vx = deg_lon_to_meters(target[1] - a.lon, a.lat)
                        norm = math.hypot(vx, vy) + 1e-12
                        ux, uy = vx/norm, vy/norm
                    else:
                        # Случайное направление, если добыча не найдена или не в радиусе
                        theta = random.random() * 2 * math.pi
                        ux, uy = math.cos(theta), math.sin(theta)

                    # Случайное отклонение от основного направления (хаос)
                    chaos_factor = 0.5 # 0 = прямолинейно, 1 = полностью хаотично
                    random_theta = random.random() * 2 * math.pi
                    rx, ry = math.cos(random_theta), math.sin(random_theta)

                    # Комбинируем направления (основное + хаос)
                    combined_x = (1 - chaos_factor) * ux + chaos_factor * rx
                    combined_y = (1 - chaos_factor) * uy + chaos_factor * ry
                    norm_combined = math.hypot(combined_x, combined_y) + 1e-12
                    final_ux = combined_x / norm_combined
                    final_uy = combined_y / norm_combined

                    # Вычисляем потенциальную новую позицию
                    vx_m = final_ux * step_m
                    vy_m = final_uy * step_m
                    dlat = meters_to_deg_lat(vy_m)
                    dlon = meters_to_deg_lon(vx_m, a.lat)
                    new_lat = a.lat + dlat
                    new_lon = a.lon + dlon

                    # Проверяем возможность перемещения
                    current_point = Point(a.lon, a.lat)
                    new_point = Point(new_lon, new_lat)
                    move_prob = get_move_probability(self.obstacles, current_point, new_point)

                    if random.random() <= move_prob:
                        # Перемещение разрешено
                        final_lat, final_lon = new_lat, new_lon
                    else:
                        # Перемещение заблокировано, остаемся на месте
                        final_lat, final_lon = a.lat, a.lon

                    # Проверка границы поля
                    if self.boundary_polygon is not None:
                        pt_check = Point(final_lon, final_lat)
                        if not self.boundary_polygon.contains(pt_check):
                            final_lat, final_lon = a.lat, a.lon # Возвращаем на место, если вышли за границу

                    a.lat, a.lon = final_lat, final_lon

            # --- Обновление Leptinotarsa ---
            alive_leptino = [a for a in self.agents_leptinotarsa if a.alive]
            for a in alive_leptino:
                a.age_days += 1
                # --- Смертность ---
                mort = BASE_MORT_LEPTINO.get(a.stage, 0.01)
                if T > UPPER_TEMP_THRESHOLD_LEPTINO or T < LOWER_TEMP_THRESHOLD_LEPTINO:
                    mort *= 2.0
                if a.stage.startswith("larva"):
                    nearby = [x for x in alive_leptino if x != a and x.alive and haversine_m(a.lat, a.lon, x.lat, x.lon) <= DENSITY_RADIUS]
                    if len(nearby) >= DENSITY_THRESHOLD:
                        mort += len(nearby) * DENSITY_MORTALITY_FACTOR
                if a.stage.startswith("larva"):
                    nearby_predators = [p for p in alive_podisus if haversine_m(a.lat, a.lon, p.lat, p.lon) <= PREY_DETECTION_RADIUS_PODISUS]
                    if nearby_predators:
                        if random.random() < 0.5:
                            a.alive = False
                            continue
                if random.random() < mort:
                    a.alive = False
                    continue

                # --- Развитие ---
                a.progress(dd_since_year_start)

                # --- Размножение (только после начала симуляции) ---
                if a.stage == "adult" and a.sex == "F" and a.age_days >= MIN_AGE_ADULT_LEPTINO:
                    if random.random() < 0.05:
                        for _ in range(CLUTCH_SIZE_LEPTINO):
                            new_lat = a.lat + meters_to_deg_lat(random.uniform(-1, 1))
                            new_lon = a.lon + meters_to_deg_lon(random.uniform(-1, 1), a.lat)
                            dd_at_birth_new = dd_since_year_start
                            self.agents_leptinotarsa.append(LeptinotarsaIndividual(new_lat, new_lon, "egg", dd_at_birth=dd_at_birth_new))

                # --- Движение ---
                step_m = MOVE_BASE_METERS_LEPTINO.get(a.stage, 0.0)
                if step_m > 0:
                    # --- НОВОЕ: Хаотичное движение для жука ---
                    chaos_factor = 0.7 # Жуки более хаотичны
                    theta = random.random() * 2 * math.pi
                    ux, uy = math.cos(theta), math.sin(theta)

                    # Случайное отклонение (хаос)
                    random_theta = random.random() * 2 * math.pi
                    rx, ry = math.cos(random_theta), math.sin(random_theta)

                    # Комбинируем
                    combined_x = (1 - chaos_factor) * ux + chaos_factor * rx
                    combined_y = (1 - chaos_factor) * uy + chaos_factor * ry
                    norm_combined = math.hypot(combined_x, combined_y) + 1e-12
                    final_ux = combined_x / norm_combined
                    final_uy = combined_y / norm_combined

                    # Вычисляем потенциальную новую позицию
                    vx_m = final_ux * step_m
                    vy_m = final_uy * step_m
                    dlat = meters_to_deg_lat(vy_m)
                    dlon = meters_to_deg_lon(vx_m, a.lat)
                    new_lat = a.lat + dlat
                    new_lon = a.lon + dlon

                    # Проверяем возможность перемещения
                    current_point = Point(a.lon, a.lat)
                    new_point = Point(new_lon, new_lat)
                    move_prob = get_move_probability(self.obstacles, current_point, new_point)

                    if random.random() <= move_prob:
                        final_lat, final_lon = new_lat, new_lon
                    else:
                        final_lat, final_lon = a.lat, a.lon

                    # Проверка границы поля
                    if self.boundary_polygon is not None:
                        pt_check = Point(final_lon, final_lat)
                        if not self.boundary_polygon.contains(pt_check):
                            final_lat, final_lon = a.lat, a.lon

                    a.lat, a.lon = final_lat, final_lon

            # Сохранение истории
            day_points_podisus = [(a.lat, a.lon, a.stage) for a in self.agents_podisus if a.alive]
            day_points_leptino = [(a.lat, a.lon, a.stage) for a in self.agents_leptinotarsa if a.alive]

            self.history_podisus.append(day_points_podisus)
            self.history_leptinotarsa.append(day_points_leptino)

            self.export_geojson_day(day, day_points_podisus, "podisus")
            self.export_geojson_day(day, day_points_leptino, "leptinotarsa")
            self.export_geotiff_day(day, day_points_podisus, "podisus")
            self.export_geotiff_day(day, day_points_leptino, "leptinotarsa")

    def export_geojson_day(self, d, pts, species_name):
        feats = []
        for (lat, lon, stg) in pts:
            feats.append({
                "type":"Feature",
                "geometry":{"type":"Point","coordinates":[lon, lat]},
                "properties":{"stage":stg, "species": species_name}
            })
        fc = {"type":"FeatureCollection", "features":feats}
        filename = f"simulation_output/{species_name}_points_day_{d:03d}.geojson"
        with open(filename, "w", encoding="utf-8") as f:
            json.dump(fc, f, ensure_ascii=False, indent=2)

    def export_geotiff_day(self, d, pts, species_name):
        if not pts:
            return
        lats = [p[0] for p in pts]; lons = [p[1] for p in pts]
        min_lat, max_lat = min(lats), max(lats); min_lon, max_lon = min(lons), max(lons)
        pad = 20.0
        padlat = meters_to_deg_lat(pad); padlon = meters_to_deg_lon(pad, (min_lat+max_lat)/2.0)
        min_lat -= padlat; max_lat += padlat; min_lon -= padlon; max_lon += padlon

        center_lat = (min_lat + max_lat)/2.0
        x_min_m = deg_lon_to_meters(min_lon, center_lat)
        x_max_m = deg_lon_to_meters(max_lon, center_lat)
        y_min_m = deg_lat_to_meters(min_lat)
        y_max_m = deg_lat_to_meters(max_lat)
        width_m = x_max_m - x_min_m; height_m = y_max_m - y_min_m
        if width_m <= 0 or height_m <= 0:
            return
        px = max(1, int(math.ceil(width_m / PIXEL_SIZE_M)))
        py = max(1, int(math.ceil(height_m / PIXEL_SIZE_M)))

        pix_h = meters_to_deg_lat(PIXEL_SIZE_M); pix_w = meters_to_deg_lon(PIXEL_SIZE_M, center_lat)
        transform = from_origin(min_lon, max_lat, pix_w, pix_h)

        raster = np.zeros((py, px), dtype=np.uint16)
        for lat, lon, _ in pts:
            col = int((lon - min_lon) / pix_w)
            row = int((max_lat - lat) / pix_h)
            if 0 <= row < py and 0 <= col < px:
                raster[row, col] += 1

        filename = f"simulation_output/{species_name}_density_day_{d:03d}.tif"
        with rasterio.open(filename, 'w',
                           driver='GTiff', height=py, width=px,
                           count=1, dtype=raster.dtype,
                           crs="EPSG:4326", transform=transform, compress='lzw') as dst:
            dst.write(raster, 1)




In [ ]:
# ---------------------------
#  Параметры симуляции
# ---------------------------

# Инициализация начальных точек и количества особей (Podisus)
INIT_POINTS = [(52.134873, 79.278759)]
INIT_COUNTS_PODISUS = {"egg": 0, "nymph1": 0, "nymph2": 0, "nymph3": 0, "nymph4": 0, "nymph5": 0, "adult": 100}
INIT_COUNTS_LEPTINOTARSA = {"egg": 0, "larva1": 0, "larva2": 0, "larva3": 0, "larva4": 0, "pupa": 0, "adult": 100}

SIM_DAYS = 30

# --- Параметры Podisus maculiventris (хищный клоп) ---
# DD (Degree Days)
NYMPH_DD_RANGE_PODISUS = (233.9, 338.7)
EGG_DD_PODISUS = 50.0
BASE_TEMP_EGG_PODISUS = 10.7
BASE_TEMP_NYMPH_PODISUS = 11.7
NYMPH_STAGE_PROP_PODISUS = np.array([0.10, 0.18, 0.20, 0.25, 0.27])
BASE_MORT_PODISUS = {
    "egg": 0.01, "nymph1": 0.01, "nymph2": 0.007, "nymph3": 0.005, "nymph4": 0.005, "nymph5": 0.005, "adult": 0.002
}
MOVE_BASE_METERS_PODISUS = {
    "egg": 0.0, "nymph1": 0.5, "nymph2": 2.0, "nymph3": 6.0, "nymph4": 10.0, "nymph5": 15.0, "adult": 25.0
}
PREY_DETECTION_RADIUS_PODISUS = 10.0
PREY_BIAS_PODISUS = 0.8 # <- Уменьшено для большей хаотичности
UPPER_TEMP_THRESHOLD_PODISUS = 35.0
LOWER_TEMP_THRESHOLD_PODISUS = 10.0
MIN_AGE_ADULT_PODISUS = 5

# --- Параметры Leptinotarsa decemlineata (колорадский жук) ---
BASE_TEMP_EGG_LEPTINO = 11.5
BASE_TEMP_LARVA_LEPTINO = 11.5
BASE_TEMP_PUPA_LEPTINO = 11.5
BASE_TEMP_ADULT_LEPTINO = 11.5
DD_EGG_LEPTINO = 40.0
DD_LARVA_TOTAL_LEPTINO = 150.0
DD_PUPA_LEPTINO = 100.0
DD_PER_LARVA_STAGE_LEPTINO = DD_LARVA_TOTAL_LEPTINO / 4
BASE_MORT_LEPTINO = {
    "egg": 0.05, "larva1": 0.1, "larva2": 0.1, "larva3": 0.1, "larva4": 0.1, "pupa": 0.05, "adult": 0.005
}
MOVE_BASE_METERS_LEPTINO = {
    "egg": 0.0, "larva1": 0.1, "larva2": 0.2, "larva3": 0.5, "larva4": 1.0, "pupa": 0.0, "adult": 10.0
}
CONSUMPTION_LARVA_LEPTINO = [0.24, 0.50, 2.04, 4.15] # см2/день по стадиям 1-4
CONSUMPTION_ADULT_LEPTINO = 10.0 # см2/день
MIN_AGE_ADULT_LEPTINO = 5
CLUTCH_SIZE_LEPTINO = 30
FECUNDITY_LEPTINO = 10
UPPER_TEMP_THRESHOLD_LEPTINO = 33.0
LOWER_TEMP_THRESHOLD_LEPTINO = 10.0

# --- Общие параметры ---
PIXEL_SIZE_M = 1.0
DENSITY_RADIUS = math.sqrt(0.1 / math.pi)
DENSITY_THRESHOLD = 3
DENSITY_MORTALITY_FACTOR = 0.1

# --- НОВОЕ: Дата начала года для DD ---
YEAR_START_DATE = datetime.date(2024, 1, 1) # <- Задается вручную
MANUAL_PREY_POINTS = [
    (52.135000, 79.279000, {"larva1": 50, "larva2": 30, "adult": 20}),
    (52.134500, 79.278500, {"larva3": 40, "larva4": 60}),
]

# --- Путь к файлу с преградами ---
BORDER_GEOJSON_PATH = r"C:\Users\pochy\Desktop\Стартап Экологические услуги\Модель расселения\border.geojson"


# ---------------------------
#  Основной запуск
# ---------------------------

if __name__ == "__main__":
    weather_file = r"C:\Users\pochy\Desktop\Стартап Экологические услуги\Модель расселения\wr352396a1\wr352396a1.txt"
    sim_start_date_str = "15.06.2024"
    sim = CombinedSimulator(
        init_points = INIT_POINTS,
        init_counts_podisus = INIT_COUNTS_PODISUS,
        init_counts_leptinotarsa = INIT_COUNTS_LEPTINOTARSA,
        sim_days = SIM_DAYS,
        weather_filepath = weather_file,
        weather_start_date_str = sim_start_date_str,
        manual_prey_points=MANUAL_PREY_POINTS,
        year_start_date=YEAR_START_DATE,
        border_geojson_path=BORDER_GEOJSON_PATH, # Передаем путь к преградам
        geojson_path = None
    )
    sim.run()
    print("Done. Outputs in simulation_output/")


Simulating days: 100%|██████████| 30/30 [00:11<00:00,  2.71it/s]

Done. Outputs in simulation_output/
